# Covid-19 and the environment, a statistics case study

The purpose of this notebook is to discover potential links between the awareness of the environment and the actual state of the environment around the time of the Covid-19 lockdowns.
- First, we explore the dataset that was given in the problem statement. We merge, filter and prepare the data for further use.
- Then, we present the dataset which we introduced ourselves : more data from Wikipedia (nicknamed "precise wikipedia) the World Air Quality index and the plastics production statistics from Eurostat.
- Then, we analyze the links between wikipedia views with indicator of actual pollution.
- Finally, we present a statistical extrapolation of environment data without Covid and compare it to the actual data, to give an idea of what might have happened if Covid hadn't happened.

# Part 1 : Coronawiki

## Imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
import os
import copy
from statsmodels.stats import diagnostic
from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
import datetime
from datetime import timedelta
from dateutil.relativedelta import relativedelta
from statsmodels.tsa.seasonal import seasonal_decompose
import statsmodels
from sklearn.metrics import mean_absolute_error
from matplotlib.pyplot import figure
from dtw import dtw,accelerated_dtw
from statsmodels.stats import diagnostic
from statsmodels.tsa.stattools import grangercausalitytests
from statsmodels.stats.multitest import fdrcorrection
np.random.seed(0)
np.random.RandomState(0)

## Interventions

In this data, each language is mapped to the main country of usage except for English, where the language's usage is very high in multiple countries such that it couldn't be mapped to a single country. As such, for that language, we have most of the data missing.

Per country, the pandemic timeline is represented, such as the first registered case, the first death, etc.

Note that the paper says that nine languages are spoken in a single language, but we have more than that here (reason for that unknown).

In [ ]:
interventions = pd.read_csv("Data/interventions.csv")


def transform_column(column_name):
    interventions[column_name] = pd.to_datetime(interventions[column_name])


for intervention in interventions.columns[1:]:
    transform_column(intervention)
interventions

## Timeseries <a id='timeseries'></a>

The most important data we have in this dataset are time series of the Wikipedia views from 2018 to July 2020 for 14 different languages: one part are the total views for all of that language's wikipedia, a second part are the views for the articles that are related to Covid-19, as well as the percentage. Finally, we also have for the same window of time the views for different topics.

In [ ]:
timeseries = pd.read_json("Data/aggregated_timeseries.json.gz")
timeseries.head()

In [ ]:
timeseries.columns

In [ ]:
languages_unique = timeseries.columns.map(lambda x: x[:2]).unique()

Correspondence:
- ja -> Japanese
- it -> Italian
- da -> Danish
- tr -> Turkish
- no -> Norwegian
- en -> English
- sr -> Serbian
- sv -> Swedish
- nl -> Dutch
- de -> German
- fr -> French
- ca -> Catalan
- ko -> Korean
- fi -> Finnish

### Splitting the timeseries data into different dataframes

As we can see, the data's format isn't ideal: for each language, the data is split into 3 Python dictionaries corresponding to the data described above, and it would be nice to separate these pieces of data to be able to read directly for each date, for example, the total number of views accross all languages, instead of having to iterate over each language's dictionnary every time.

This will also make the analysis phase easier later on.

### Total sum of views, views of articles related to Covid

<a id='extractionformat'></a>
In this part of the code we extract two following kind of data, for each date:
- For every language's Wikipedia, the total number of views on that particular date
- For every language's Wikipedia, the total number of views for articles related to Covid-19 on that particular date

---


Every resulting dataframe will have the following format:

| Column name          | Description                                                                                                                                                                                       |   |   |   |
|----------------------|---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|---|---|---|
| date           | A particular date between January 2018 (inclusive) and July of 2020 (inclusive)                                                                                                                         |   |   |   |                    
| language       | The corresponding wikipedia language                                                                |   |   |   |
| views  | The number of views (total or only for articles related to Covid-19) for this language and date



---



We also extract another dataframe that simply maps for each language the number of articles that were considered in the original experiment. Finally, a last dataframe which is redundant with the two first ones is present: one which gives for each day the percentage of views that went to Covid related articles.

In [ ]:
timeseries_total_sum_dict = {}
timeseries_covid_len_dict = {}
timeseries_covid_sum_dict = {}
timeseries_covid_percent_dict = {}
for cn in timeseries.columns:
    timeseries_total_sum_dict[cn] = timeseries[cn]["sum"]
    timeseries_covid_len_dict[cn] = timeseries[cn]["covid"]["len"]
    timeseries_covid_sum_dict[cn] = timeseries[cn]["covid"]["sum"]
    timeseries_covid_percent_dict[cn] = timeseries[cn]["covid"]["percent"]

In [ ]:
sum_data_df = pd.DataFrame.from_dict(timeseries_total_sum_dict, orient="index").T
covid_len_data_df = pd.DataFrame.from_dict(
    timeseries_covid_len_dict, orient="index", columns=["len"]
).T
covid_sum_data_df = pd.DataFrame.from_dict(timeseries_covid_sum_dict, orient="index").T
covid_percent_data_df = pd.DataFrame.from_dict(
    timeseries_covid_percent_dict, orient="index"
).T

In [ ]:
# Fuse the .m and the normal columns together, and separate the data
new_sum_data_df = pd.DataFrame()
new_covid_sum_data_df = pd.DataFrame()
for language in languages_unique:
    country_sum_data = pd.DataFrame()
    country_sum_data["views"] = sum_data_df[language] + sum_data_df[language + ".m"]
    country_sum_data["language"] = language
    new_sum_data_df = pd.concat([new_sum_data_df, country_sum_data], axis=0)

    country_covid_sum_data = pd.DataFrame()
    country_covid_sum_data["views"] = (
        covid_sum_data_df[language] + covid_sum_data_df[language + ".m"]
    )
    country_covid_sum_data["language"] = language
    new_covid_sum_data_df = pd.concat(
        [new_covid_sum_data_df, country_covid_sum_data], axis=0
    )

sum_data_df = new_sum_data_df
covid_sum_data_df = new_covid_sum_data_df

In [ ]:
sum_data_df.index = covid_sum_data_df.index = pd.to_datetime(sum_data_df.index)
covid_percent_data_df.index = pd.to_datetime(covid_percent_data_df.index)
sum_data_df["date"] = covid_sum_data_df["date"] = sum_data_df.index
covid_percent_data_df["date"] = covid_percent_data_df.index
# covid_sum_data_df = covid_sum_data_df[new_column_order]
# covid_percent_data_df = covid_percent_data_df[new_column_order]

In [ ]:
sum_data_df.head()

In [ ]:
covid_percent_data_df.head()

### Checking for missing data

Before continuing further, let us check for missing data in the timeseries; this will help us avoid bad surprises later on.

In [ ]:
sum_data_df.isnull().any().any(), covid_sum_data_df.isnull().any().any(), covid_percent_data_df.isnull().any().any()

There appears to be some missing data in the percentage dataframe; let's check by language where that missing data is reported.

In [ ]:
missing_data_language = covid_percent_data_df.isnull().any(axis=0)
missing_data_language[missing_data_language]

Looking at the paper, this corresponds to Swedish.

In [ ]:
# Decomment Run this cell if you want to see how the missing data can be seen in the original timeseries
# timeseries.loc[:,'sv']['sum']

It appears that, for desktop devices, the views for the Swedish Wikipedia haven't been collected for the whole year 2018. The reason for that is unknown, as a quick search tells us that this version has existed since 2001. The mobile data, however, is available. We will need to take this into account when doing our analysis in the future, for example by not taking into account 2018.

### Topics data

Now we will extract for each language, all the views per topic in such a way that the data becomes more usable. In the original data, all topic-related information was in a single dictionnary; we're gonna separate them in a way that each column will correspond to a different topic, with each row being a different language.

In [ ]:
country_to_topics = {}
for cn in timeseries.columns:
    country_to_topics[cn] = timeseries[cn]["topics"]
topics_df = pd.DataFrame.from_dict(country_to_topics, orient="index")

In [ ]:
countries_to_topics_len = {}
countries_to_topics_sum = {}
countries_to_topics_percent = {}
for country in topics_df.index:
    countries_to_topics_len[country] = {}
    countries_to_topics_sum[country] = {}
    countries_to_topics_percent[country] = {}
    for topic in topics_df.columns:
        countries_to_topics_len[country][topic] = topics_df.loc[country, topic]["len"]
        countries_to_topics_sum[country][topic] = topics_df.loc[country, topic]["sum"]
        countries_to_topics_percent[country][topic] = topics_df.loc[country, topic][
            "percent"
        ]
countries_to_topics_len_df = pd.DataFrame.from_dict(
    countries_to_topics_len, orient="index"
)
countries_to_topics_sum_df = pd.DataFrame.from_dict(
    countries_to_topics_sum, orient="index"
)
countries_to_topics_percent_df = pd.DataFrame.from_dict(
    countries_to_topics_percent, orient="index"
)

In [ ]:
countries_to_topics_sum_df.head()

In [ ]:
# Decomment and run this cell to see all available topics.
# countries_to_topics_sum_df.columns

However, we are not be interested in all available topics. As a matter of fact, for our project, it is only useful to isolate the data about articles related to the environment. Examining the columns, the topic is available in only one of them, so we will extract only that topic in two dataframes that have the same format as [here](#extractionformat) (only difference is that we change the name *views* to *environment_views*).

In [ ]:
sum_environment_df = countries_to_topics_sum_df["STEM.Earth and environment"]
percent_environment_df = countries_to_topics_percent_df["STEM.Earth and environment"]
country_to_env_data_sum = {}
country_to_env_data_percent = {}
for country in sum_environment_df.index:
    country_to_env_data_sum[country] = sum_environment_df[country]
    country_to_env_data_percent[country] = percent_environment_df[country]
sum_environment_df = pd.DataFrame.from_dict(country_to_env_data_sum, orient="index").T
percent_environment_df = pd.DataFrame.from_dict(
    country_to_env_data_percent, orient="index"
).T
percent_environment_df.index = pd.to_datetime(percent_environment_df.index)

new_sum_environment_df = pd.DataFrame()
for language in languages_unique:
    country_env_sum_data = pd.DataFrame()

    country_env_sum_data["environment_views"] = (
        sum_environment_df[language] + sum_environment_df[language + ".m"]
    )
    country_env_sum_data["language"] = language
    new_sum_environment_df = pd.concat(
        [new_sum_environment_df, country_env_sum_data], axis=0
    )
sum_environment_df = new_sum_environment_df
sum_environment_df.index = pd.to_datetime(sum_environment_df.index)
sum_environment_df["date"] = sum_environment_df.index

In [ ]:
sum_environment_df

In [ ]:
# Not really useful piece of data
percent_environment_df.head()

### Plots

In the following part, we save the following type of plots:

- per language on Wikipedia, three separate line plots showing the evolution of the total number of views, the number of views of articles related to Covid-19, and the number of views of environment articles.
- per language on Wikipedia, three separate histograms representing the number of views per day, for the same categories as in the point above.

These plots can be found in the 'Figures' folder that will be created after the launch of all cells in this part, but we will plot some examples in the notebook to try and detect some patterns.

In [ ]:
figures_path = "./Figures/"
timeseries_path = "timeseries/"
hists_path = "hists/"
total_views_path = "all_views/"
covid_views_path = "covid_views/"
topic_views_path = "topic_views/"


def make_sub_dirs(main_dir):
    os.mkdir(main_dir + total_views_path)
    os.mkdir(main_dir + covid_views_path)
    os.mkdir(main_dir + topic_views_path)
    os.mkdir(main_dir + topic_views_path + total_views_path)
    os.mkdir(main_dir + topic_views_path + covid_views_path)

In [ ]:
if not os.path.exists(figures_path):
    os.mkdir(figures_path)
    os.mkdir(figures_path + timeseries_path)
    os.mkdir(figures_path + hists_path)
    make_sub_dirs(figures_path + timeseries_path)
    make_sub_dirs(figures_path + hists_path)

In [ ]:
def lineplot_language_views_timeseries(data, country, covid_views=False):
    fig, ax = plt.subplots(figsize=(10, 6), dpi=100)
    filtered_data = data[data["language"] == country]
    x = filtered_data.index
    y = filtered_data.views
    g = sns.lineplot(x=x, y=y)
    plt.xticks(fontsize=8)
    g.set(xlabel="Dates")
    if covid_views:
        title = "Wikipedia page views for articles related to Covid-19 for {}".format(
            country
        )
    else:
        title = "Wikipedia page views for {}".format(country)
    g.set(ylabel="Page views", title=title)
    # ax.set(xscale="log")
    if covid_views:
        plt.savefig(figures_path + timeseries_path + covid_views_path + title + ".jpg")
    else:
        plt.savefig(figures_path + timeseries_path + total_views_path + title + ".jpg")
    plt.close(fig)
    # plt.show()

In [ ]:
def hist_language_views(data, country, covid_views=False):
    fig, ax = plt.subplots(figsize=(10, 6), dpi=100)
    filtered_data = data[data["language"] == country]
    g = sns.histplot(data=filtered_data, x="views", bins=50)
    if covid_views:
        title = (
            "Wikipedia views distribution for articles related to Covid-19 {}".format(
                country
            )
        )
    else:
        title = "Wikipedia views distribution for {}".format(country)
    g.set(title=title)
    if covid_views:
        plt.savefig(figures_path + hists_path + covid_views_path + title + ".jpg")
    else:
        plt.savefig(figures_path + hists_path + total_views_path + title + ".jpg")

    plt.close(fig)

In [ ]:
for country_code in sum_data_df.language.unique():
    lineplot_language_views_timeseries(sum_data_df, country_code)
    hist_language_views(sum_data_df, country_code)
    lineplot_language_views_timeseries(covid_sum_data_df, country_code, True)
    hist_language_views(covid_sum_data_df, country_code, True)

In [ ]:
def lineplot_topic_views_timeseries(
    topic_data, country, covid_views=False, topic="environment"
):
    fig, ax = plt.subplots(figsize=(10, 6), dpi=100)
    filtered_data = topic_data[topic_data["language"] == country]
    x = filtered_data.index
    y = filtered_data.environment_views
    g = sns.lineplot(x=x, y=y)
    plt.xticks(fontsize=8)
    g.set(xlabel="Dates")
    if covid_views:
        title = "Wikipedia page views for articles related to Covid-19 for {0} for the {1} topic".format(
            country, topic
        )
    else:
        title = "Wikipedia page views for {0} for the {1} topic".format(country, topic)
    g.set(ylabel="Page views", title=title)
    if covid_views:
        plt.savefig(
            figures_path
            + timeseries_path
            + topic_views_path
            + covid_views_path
            + title
            + ".jpg"
        )
    else:
        plt.savefig(
            figures_path
            + timeseries_path
            + topic_views_path
            + total_views_path
            + title
            + ".jpg"
        )

    plt.close(fig)

In [ ]:
def hist_topic_views(topic_data, country, covid_views=False, topic="environment"):
    fig, ax = plt.subplots(figsize=(10, 6), dpi=100)
    filtered_data = topic_data[topic_data["language"] == country]
    g = sns.histplot(data=filtered_data, x="environment_views", bins=50)
    if covid_views:
        title = "Wikipedia views distribution for articles related to Covid-19 for {0} for the {1} topic".format(
            country, topic
        )
    else:
        title = "Wikipedia views distribution for {0} for the {1} topic".format(
            country, topic
        )
    g.set(title=title)
    if covid_views:
        plt.savefig(
            figures_path
            + hists_path
            + topic_views_path
            + covid_views_path
            + title
            + ".jpg"
        )
    else:
        plt.savefig(
            figures_path
            + hists_path
            + topic_views_path
            + total_views_path
            + title
            + ".jpg"
        )

    plt.close(fig)

In [ ]:
for country_code in sum_environment_df.language.unique():
    lineplot_topic_views_timeseries(sum_environment_df, country_code)
    hist_topic_views(sum_environment_df, country_code)

Let us plot some of the environment views timeseries here, to try and see if some interesting patterns can be seen already. We will choose the Serbian and Danish languages for this part. Note that we're not comparing the values themselves but whether or not there are patterns in either of them, so sharing the y-axis isn't really needed here.

In [ ]:
countries_chosen = ["sr", "da"]
fig, ax = plt.subplots(nrows=1, ncols=len(countries_chosen), figsize=(20, 10), dpi=100)
for i in range(len(countries_chosen)):
    filename = "Wikipedia page views for {} for the environment topic.jpg".format(
        countries_chosen[i]
    )
    ax[i].imshow(
        plt.imread(fname="./Figures/timeseries/topic_views/all_views/" + filename)
    )
    ax[i].axis("off")
# _ = plt.axis('off')

We can see a slight difference in behavior between these languages around the beginning of 2020:
- first, we can see that while the environment views are pretty low before 2020 for the serbian wikipedia, these views go up quickly around the end of March 2020, which when looking at the intervention data seems to coincide with the first serbian lockdown. It is still a bit early however to affirm that one of these elements caused the other.
- moving to the danish wikipedia however, there doesn't seem to be a particular rise or fall in the number of environment views.

We can also see however that for both languages, there seems to be a pattern of evolution of views between the years: for serbian, there is clear decrease between June and Septembre of both 2018 and 2019, which seems to have happened in 2020 as well if we look at the left plot. For danish the same phenomena can be observed as well, this time between July and August of both 2018 and 2019; again, this seems to have happened in 2020 as well if we were to look at the right plot. Looking at the other languages, patterns can also be found accross years.

It appears that indeed, the number of views for environmental articles did change in the same timeframe as when Covid-19 first arrived in some of the countries. It's equally important, however, to state that some don't display a change in pattern, and that at this point in the analysis **we can't say** that Covid-19 indeed caused more or less environment awareness on Wikipedia.

### Statistically testing if there was indeed a difference between 2019 and 2020 <a id='firststat'></a>

Let us now do another, more statistical analysis; we will test the hypothesis that, in average, and for every language, the average number of environmental views is the same between 2019 (pre-Covid) and 2020 (during the first, biggest wave of Covid). We will test these hypotheses using the $\alpha = 0.05 $ significance level, as well as using the Bonferonni correction $\alpha_{c} = \frac{\alpha}{n}$, with n = 14 .

We match the same dates between 2019 and 2020 (up to the 31st of July, as that's when the Coronawiki data ends) for every language, and conduct a t-test for the null hypothesis described above. Note that we consider the samples to be related, as they come from the same language and for the same range of dates. In this setup, a negative t-statistic means that for that language, there are more environment views on average in 2020, and vice-versa.

In [ ]:
before_covid_env_topic_views = sum_environment_df[
    (sum_environment_df.date < "2020-01-01") & (sum_environment_df.date >= "2019-01-01")
]
during_covid_env_topic_views = sum_environment_df[
    sum_environment_df.date >= "2020-01-01"
]
for language in sum_environment_df.language.unique():
    language_before = before_covid_env_topic_views[
        before_covid_env_topic_views.language == language
    ].copy()
    language_during = during_covid_env_topic_views[
        during_covid_env_topic_views.language == language
    ].copy()

    language_before.date = language_before.date.apply(
        lambda date: str(date.month) + "-" + str(date.day)
    )
    language_during.date = language_during.date.apply(
        lambda date: str(date.month) + "-" + str(date.day)
    )
    matching = pd.merge(
        language_before,
        language_during,
        on=["date", "language"],
        suffixes=["_before", "_during"],
    )
    stat, pvalue = stats.ttest_rel(
        matching["environment_views_before"], matching["environment_views_during"]
    )
    print("p-value for {0}: {1}".format(language, pvalue))
    if pvalue >= 0.05:
        print("Don't reject null when alpha = 0.05")
    if pvalue >= (0.05 / 14):
        print("Don't reject null when using the banferoni correction")
    print("Ttest statistic value for {0}: {1}".format(language, stat))
    print("------------------")

It appears that for 11 out of 14 languages, there is indeed a change in how people visit these pages; the null hypothesis is rejected even after applying the Bonferonni correction, seeing how small their p-values are.. Out of these 11 languages, 6 observe a negative t-statistic, i.e. an average increase in 2020, while the other 5 observe a positive t-statistic, i.e an average decrease in 2020.

## Mobility data

The second type of data we have are mobility data that come from two different sources. The first one is from Apple, who stopped giving out the data in April 2022, and the second one is from Google, which is still available, and more up-to-date (17th of October).

### Apple mobility

In [ ]:
apple_mobility = pd.read_csv("Data/applemobilitytrends-2020-04-20.csv.gz")
apple_mobility.head()

In [ ]:
print(apple_mobility.transportation_type.unique())  # Three types of transportation
print(apple_mobility.geo_type.unique())  # Granularity

The mobility data from Apple we have begins in mid-January 2020, and ends that same year in April. This isn't a big time window, and it doesn't appear that there is earlier data as it has been collected specifically for Covid-19 mobility tracking.

Three types of transportation have been tracked here: driving, walking, and transit. We also have two different granularities about the collected data: either country/world region level, or city level, which are often country capitals.

Per day and region, we have the pourcentage of the usage of every transportation mode according to some pre-pandemic baseline computed in early 2020.

In [ ]:
apple_mobility.isnull().any().any()  # There doesn't appear to be null data, but verifying shows that some countries don't have all the transportation types (transit, mostly)

In [ ]:
# Convert dates strings to date times
time_columns = pd.to_datetime(apple_mobility.columns[3:])
apple_mobility.columns = apple_mobility.columns[:3].append(time_columns)

### Global mobility from Google

In [ ]:
global_mobility_report = pd.read_csv("Data/Global_Mobility_Report.csv.gz")

In [ ]:
print(
    min(global_mobility_report.date.unique()), max(global_mobility_report.date.unique())
)
global_mobility_report.date = pd.to_datetime(global_mobility_report.date)

The mobility data from Google we have begins in mid-February 2020, and ends that same year in August. This is more than the given Apple data, despite the fact that both collections happened in the context of Covid-19.

There are more levels of granularity with this data: for example, for the United Arab Emirates, we might simply talk about the whole country, or it could be specified in the column *sub_region_1* that the row is actually focused on the city of Abu Dhabi. This granularity can be made finer with *sub_region_2*.

Per day and region, we have the **difference** in pourcentage usage of various location types (workplaces, etc) according to some pre-pandemic baseline computed in the early weeks of 2020. The baseline was computed *per day* , as people can have different behaviors depending on whether it's the weekend or not.

In [ ]:
global_mobility_report.isnull().sum() / global_mobility_report.shape[0]

As expected, we have more coarse grained data (no missing data) than finer grained (many sub_region_1 fields are null, and even more sub_region_2 as well). The metropolitan area is very rarely defined, as almost 99.4% of the field is empty values.

Looking at the differences from baseline, we remark scarcity as well; apart from workplace locations which has a missing rate of only around 5.04%, others go from 36.4% (for retail) to 53.7% (for parks).

We can look in more details at the entries which have the missing values for the differences from baseline; let's check the intersection of these missing values, to see for example if the absence of one field implies the absence of the others.

In [ ]:
retail_missing = (
    global_mobility_report.retail_and_recreation_percent_change_from_baseline.isnull()
)
grocery_pharmecy_missing = (
    global_mobility_report.grocery_and_pharmacy_percent_change_from_baseline.isnull()
)
park_missing = global_mobility_report.parks_percent_change_from_baseline.isnull()
transit_stations_missing = (
    global_mobility_report.transit_stations_percent_change_from_baseline.isnull()
)
workplace_missing = (
    global_mobility_report.workplaces_percent_change_from_baseline.isnull()
)
residential_missing = (
    global_mobility_report.residential_percent_change_from_baseline.isnull()
)

In [ ]:
all_missing = (
    retail_missing
    & grocery_pharmecy_missing
    & park_missing
    & transit_stations_missing
    & workplace_missing
    & residential_missing
)

In [ ]:
all_missing.any()

From the above result, we can conclude that there doesn't appear to be a feature such that if that one is missing, then all the others are missing; this also means that for each entry, there's always at least one feature available.

## Topics

Simply maps each considered article to the topics it is related to. A single article can be mapped to multiple topics. The number of articles per topic can be found in the [timeseries](#timeseries) data.

In [ ]:
# loading the data takes a lot of time, so decomment if you want to run this cell.
# topics_linked = pd.read_csv("topics_linked.csv.xz")
# topics_linked

## Regression Analysis

The statistical testing [here](#firststat) showed us that indeed, there was a difference in environment views between 2019 and 2020. However, some questions remain: 
- are these differences correlated with the rest of the Coronawiki data, such as precise mobility changes (walking, driving, etc)?
- can we explain how these views evolve depending on different time periods for each language?

In order to answer these questions, we will proceed with two different regression analyses, both trying to explain the logarithm of the environment views. We are taking the logarithm because the range of pageviews vary greatly between languages. This also makes the models multiplicative, making comparaison easier between languages (a coefficient will now correspond to a multiplicative effect on the page views). The two regression models used are the following:
- first, we will try to see if using mainly the mobility data alone (without the context of a time period) is enough to explain the (log)pageviews. There isn't a concept of treatment or control group here, it is simply a regression task.
- the second model used is more refined: here, we take two different time periods (based on the mobility data) over 2019 and 2020, effectively creating a treatment and control group according to the time period, to see if we can detect a shift in pageviews volume based on the year and the time periods. More details will be given in the [second subsection](#diffndiff)

In [ ]:
sum_environment_df['log_pages_views'] = np.log(sum_environment_df.environment_views)

### First regression: using the mobility data

#### Data merging

The data preparation phase in the first regression is more intensive as in the second, as we have to merge/standardize the format of multiple dataframes.

First of all, let us keep from both mobility datasets only the countries that were considered in the original timeseries. We mapped the english language to Washington DC, the catalan language to Barcelona, and the Korean language both to Seoul and South Korea.

In [ ]:
country_to_wiki_code = {
    "Japan": "ja",
    "Italy": "it",
    "Denmark": "da",
    "Turkey": "tr",
    "Norway": "no",
    "Serbia": "sr",
    "Sweden": "sv",
    "Netherlands": "nl",
    "Germany": "de",
    "France": "fr",
    "Barcelona": "ca",
    "South Korea": "ko",
    "Finland": "fi",
    "District of Columbia": "en",
    "Washington DC": "en",
    "Seoul": "ko",
}

In [ ]:
def map_countries_to_fine_grained(x):
    if x == "Spain":
        return "ca"
    if x == "United States":
        return "en"
    if x == "Seoul":
        return "ko"
    return country_to_wiki_code[x]


filtered_global_mobility_report = global_mobility_report[
    global_mobility_report["country_region"].isin(country_to_wiki_code)
    | global_mobility_report["sub_region_1"].isin(country_to_wiki_code)
    | global_mobility_report["sub_region_2"].isin(country_to_wiki_code)
].copy()
filtered_global_mobility_report.loc[
    :, "country_region_code"
] = filtered_global_mobility_report.country_region.apply(
    lambda x: map_countries_to_fine_grained(x)
)
filtered_global_mobility_report.index = range(len(filtered_global_mobility_report))
filtered_global_mobility_report.country_region.unique()  # There is data about all countries

In [ ]:
def add_global_mobility_language_info(cities_global_mobility_report, granularity, place, is_ankara = False):
    if not(is_ankara):
        return pd.concat(\
        [
            cities_global_mobility_report,
            filtered_global_mobility_report[
                filtered_global_mobility_report[granularity] == place
            ]
        ])
    return pd.concat(\
        [
            cities_global_mobility_report,
            filtered_global_mobility_report[
                (filtered_global_mobility_report[granularity] == place) & (filtered_global_mobility_report["sub_region_2"].isnull())
            ]
        ])

In [ ]:
cities_global_mobility_report = pd.DataFrame()
zipped = [("sub_region_1", 'Tokyo'), ("sub_region_2", 'Copenhagen Municipality'), ("sub_region_1", 'Berlin'), ("sub_region_1", 'Oslo'), ("sub_region_2", 'Barcelona'),
          ("sub_region_2", 'Helsinki'), ("sub_region_2", 'Paris'), ("sub_region_2", 'Metropolitan City of Rome'), ("metro_area", 'Seoul Metropolitan Area'),
          ("sub_region_2", 'Government of Amsterdam'), ("metro_area", 'Belgrade Metropolitan Area'), ("sub_region_2", 'Stockholm Municipality'), 
          ("sub_region_1", 'Ankara'), ("sub_region_2", 'Barcelona'), ("sub_region_1", 'District of Columbia')]

for pair in zipped:
    if pair[1] == 'Ankara':
        cities_global_mobility_report = add_global_mobility_language_info(cities_global_mobility_report, pair[0], 'Ankara', True)
    else:
        cities_global_mobility_report = add_global_mobility_language_info(cities_global_mobility_report, pair[0], pair[1])
cities_global_mobility_report = cities_global_mobility_report[
    [
        "country_region_code",
        "date",
        "retail_and_recreation_percent_change_from_baseline",
        "grocery_and_pharmacy_percent_change_from_baseline",
        "parks_percent_change_from_baseline",
        "transit_stations_percent_change_from_baseline",
        "workplaces_percent_change_from_baseline",
        "residential_percent_change_from_baseline",
    ]
]
cities_global_mobility_report = cities_global_mobility_report.rename(
    columns={"country_region_code": "language"}
).copy()

In [ ]:
filtered_apple_mobility = apple_mobility[
    apple_mobility["region"].isin(country_to_wiki_code)
].copy()
print(filtered_apple_mobility.region.unique())
print("-----")
filtered_apple_mobility.region = filtered_apple_mobility.region.apply(
    lambda x: country_to_wiki_code[x]
)
filtered_apple_mobility.head()

In [ ]:
new_apple_mobility = pd.DataFrame()
for language in sum_data_df.language.unique():
    tmp = (
        filtered_apple_mobility[filtered_apple_mobility["region"] == language]
        .T.iloc[2:]
        .copy()
    )
    tmp.columns = tmp.iloc[0]
    tmp = tmp.iloc[1:]
    tmp["language"] = language
    tmp["date"] = tmp.index
    new_apple_mobility = pd.concat([new_apple_mobility, tmp], axis=0)

tmp_merge1 = sum_data_df.merge(new_apple_mobility, on=["language", "date"])
tmp_merge2 = tmp_merge1.merge(sum_environment_df, on=["language", "date"])
final_merge = tmp_merge2.merge(cities_global_mobility_report, on=["date", "language"])
final_merge[["driving", "transit", "walking"]] = final_merge[
    ["driving", "transit", "walking"]
].astype(np.float64)

In [ ]:
final_merge.head()

There is still some null data however, so we have to manage these features, either with interpolation or filling these values with the mean of the corresponding countries.

In [ ]:
final_merge.isnull().any()

In [ ]:
final_merge[final_merge.transit.isnull()].groupby("language").count()

It appears that transit data from Apple is totally missing for the Korean, Serbian and Turkish parts; as such there is no way to replace them. Let us check for the other feature

In [ ]:
final_merge[final_merge.grocery_and_pharmacy_percent_change_from_baseline.isnull()]

We will simply interpolate these 2 missing values with the nearest value; because it's both on the same day, and that day isn't the last one per language, the nearest value will be one for the same language.

In [ ]:
final_merge[["grocery_and_pharmacy_percent_change_from_baseline"]] = final_merge[
    ["grocery_and_pharmacy_percent_change_from_baseline"]
].interpolate(method="nearest")

#### Regression analysis

We can now do regression analysis, with the logarithm of the number of environment views as output, and using the columns of *final_merge* as covariates, to see if we can fit some hyperplane to this data. First, let us transform the categorical language data using dummy variable encoding

In [ ]:
final_merge = pd.get_dummies(final_merge, drop_first=True)
final_merge.head()

Then, we will normalize the input variables, so that each column has mean 0 and standard deviation 1

In [ ]:
all_data_columns = list(final_merge.columns[2:])
all_data_columns.remove("environment_views")
#all_data_columns.remove("year")
all_data_columns.remove("log_pages_views")
for column in all_data_columns:
    final_merge[column] = (
        final_merge[column] - final_merge[column].mean()
    ) / final_merge[column].std()

#final_merge["environment_views"] = (
#    final_merge["environment_views"] - final_merge["environment_views"].mean()
#) / final_merge["environment_views"].std()

In [ ]:
formula = ""
for covariate in all_data_columns:
    formula = covariate + "+" + formula


formula = formula[: len(formula) - 1]

First we will use all covariates as predictors, with no interactions between them.

In [ ]:
mod = smf.ols(formula="""log_pages_views ~ """ + formula, data=final_merge)
res = mod.fit()
print(res.summary())

We're being told in the warnings that there either, there are strong multicollinearity problems, or the design matrix isn't invertible. This might be because of the "dummy variable trap" where, after using dummy encoding, many of the created features can be explained by the others. The condition number is also pretty high, which means that the matrix is easily perturbed by variation in the input data. We will try to fit a different model with less of the dummy variables to see if we can get rid of this issue.

We also use less data points than we have; these correspond to the three languages that don't have the Apple transit data, i.e Turkish, Serbian and Korean. We will first try to get rid of that feature for the next regression analysis

In [ ]:
features_to_exclude = [
    "transit", 
]
data_columns = list(set(all_data_columns) - set(features_to_exclude))
formula = ""
for covariate in data_columns:
    formula = covariate + "+" + formula


formula = formula[: len(formula) - 1]
# formula = formula[120:]
# formula = formula[72:]

In [ ]:
mod = smf.ols(formula="""log_pages_views ~ """ + formula, data=final_merge)
res = mod.fit()
print(res.summary())

Here, we got rid of the Apple transit feature, which makes us able to use all the data points we have. This gets rid of the warning that we used to have, meaning that we don't have multicollinearity problems anymore, and the data matrix is invertible. We can then now try to interpret the model.

The R-squared is nearly the same as before, i.e the part of explained variance is equal to 97.7% . Some of the features are not singificant, as their p-value is bigger than 0.05, so one can try to fit the model without these features.

Looking at the covariates themselves, we see that it is useful to have the language data, as obviously the range of views from one country to another can change significantly as Wikipedia's importance isn't the same everywhere.

From the Apple mobility data, it appears that the remaining features (Walking and driving) aren't statistically significant, and so it may not be suitable to use/interpret them.

For the global mobility, all but one feature (*parks_percent_change_from_baseline*) are statistically significant; however, it is quiet hard to interpret some of them. For example, it makes sense to say that an increase in the use of transit stations results in less environment views (coefficient is -0.2785) as people are using more polluting modes of transportation. Similarly, an increase in how much one stays home (the residential change) seems to indicate an increase in environment views.

However, on the flip side, other features aren't as interpretable, or appear counter-intuitive: for example, it's not really easy to explain why an increase in the visits to workplaces would cause an increase in environment views.

Performing a regression analysis using the mobility data from Apple and Google appears to explain the data quiet well; however, because it is lacking more contextualisation, it is not clear if this analysis is enough. To get more precise, we will now conduct the difference-in-difference regressions, to check more clearly the differences between not only 2019 and 2020, but also specific time periods in each year.

### Difference in Difference regression <a id='diffndiff'></a>

The second type of analysis we will now conduct is a difference-in-differences regression; as explained in the original paper, it follows the same logic as observational data analysis, as we can define a concept of treatment and control if we were to use certain time periods as points of reference.

First, we have to prepare the data; as in the paper, we will define certain time windows of (up to) 5 weeks around both the change of mobility dates, and the return to normalcy dates.  We have to do this separately for each language, and keep the same dates in 2019. 

Note that we lack the Turkish data, as by the time Coronawiki was created, Turkey still didn't have even a lockdown (which happened in April 2021).

In [ ]:
thirty_five_days = timedelta(days = 35)
## For each date, calculate the 5 week time window preceeding and following it
## We keep only the day and months so that when we define the time periods later, we can also do it for 2019.
interventions['Mobility_35_days_before'] = (interventions['Mobility'] - thirty_five_days).dt.strftime('%m-%d')
interventions['Mobility_35_days_after'] = (interventions['Mobility'] + thirty_five_days).dt.strftime('%m-%d')
interventions['Mobility'] = interventions['Mobility'].dt.strftime('%m-%d')
interventions['Normalcy_35_days_before'] = (interventions['Normalcy'] - thirty_five_days).dt.strftime('%m-%d')
interventions['Normalcy_35_days_after'] = (interventions['Normalcy'] + thirty_five_days).dt.strftime('%m-%d')
interventions['Normalcy'] = interventions['Normalcy'].dt.strftime('%m-%d')

In [ ]:
# Boolean variables that define the time periods; note that for attributes with prefix 'before_', this means that the date is in a 5 weeks time window before
# the corresponding date for a given language; for the prefix 'after_', this means that it is in the 5 weeks time window after the date for that language
sum_environment_df['before_mobility'] = 0
sum_environment_df['after_mobility'] = 0
sum_environment_df['before_normalcy'] = 0
sum_environment_df['after_normalcy'] = 0
sum_environment_df['year'] = sum_environment_df['date'].dt.year
sum_environment_df['date_without_year'] =  sum_environment_df['date'].dt.strftime('%m-%d')

In [ ]:
def set_values_for_language(mask, date_1, date_2, column_name):
    #For a given boolean variable, set to 1 if the dates are in the given time window denoted by date_1 and date_2, otherwise it's 0
    sum_environment_df.loc[(mask) & \
                        (sum_environment_df['date_without_year'] >= date_1) & \
                        (sum_environment_df['date_without_year'] < date_2), column_name] = 1

    

for language in languages_unique:
    if language == 'tr': #ignore the turkish language as we don't have mobility data for it
        continue
    
    constant_mask_interventions = (interventions['lang'] == language)
    intervention_for_that_language = interventions[constant_mask_interventions]
    
    #Keep the 3 mobility-related dates for that language
    before_mobility_date = intervention_for_that_language['Mobility_35_days_before'].item()
    mobility_date = intervention_for_that_language['Mobility'].item()
    after_mobility_date = intervention_for_that_language['Mobility_35_days_after'].item()
    
    #Keep the 3 normalcy-related dates for that language
    before_normalcy_date = intervention_for_that_language['Normalcy_35_days_before'].item()
    normalcy_date = intervention_for_that_language['Normalcy'].item()
    after_normalcy_date = intervention_for_that_language['Normalcy_35_days_after'].item()
    
    constant_mask_env = (sum_environment_df['language'] == language)
    set_values_for_language(constant_mask_env, before_mobility_date, mobility_date, 'before_mobility')
    set_values_for_language(constant_mask_env, mobility_date, after_mobility_date, 'after_mobility')
    set_values_for_language(constant_mask_env, before_normalcy_date, normalcy_date, 'before_normalcy')
    set_values_for_language(constant_mask_env, normalcy_date, after_normalcy_date, 'after_normalcy')

In [ ]:
# Cell for sanity check, no use to run it
_ = """for language in languages_unique:
    print(language)
    for c in ['before_mobility', 'after_mobility', 'before_normalcy', 'after_normalcy']:
        print(c)
        print(len(sum_environment_df[(sum_environment_df['language'] == language)&(sum_environment_df[c] == 1) & (sum_environment_df['year'] == 2020)]))
    print("------")"""

In [ ]:
sum_environment_df = sum_environment_df.drop(labels = 'date_without_year', axis = 1) # We don't need that column anymore
sum_environment_df_2019_2020 = sum_environment_df[sum_environment_df['year'] >= 2019].copy() #Keep only data from 2019 and 2020
sum_environment_df_2019_2020['is_2020'] = (sum_environment_df_2019_2020.year == 2020).astype(int) #Create a boolean variable to denote the year

Now, we will define the 3 regression models that we will build. For all of these, the data around certain dates from 2020 is considered to be the treated one, while the 2019 one is the control :
- for the first, we take the mobility change date as our only reference, by using the 10 week data around that date for each language. The goal is to quantify the immediate change after the mobility restriction
- for the second , we take the data before mobility restriction and the data after normalcy, to see if having the same mobility means having exactly the same visits to the environment pages
- for the third and last, we take the data after the mobility change and the data after the return to normalcy, to see if by returning to normal, the environment visits also changed, and how much 

In [ ]:
# before and after mobility change
sum_environment_df_2019_2020_before_after_mobility = sum_environment_df_2019_2020[(sum_environment_df_2019_2020.before_mobility == 1) |\
                             (sum_environment_df_2019_2020.after_mobility == 1)].drop(labels = ['before_normalcy', 'after_normalcy'], axis = 1)
sum_environment_df_2019_2020_before_after_mobility['period'] = (sum_environment_df_2019_2020_before_after_mobility.after_mobility ==1).astype(int)

# before mobility and after normalcy
# note that for this dataframe and the next, we have to exclude the ca language, as it doesn't have a return to normacly date.
sum_environment_df_2019_2020_before_mobility_after_normalcy = sum_environment_df_2019_2020[(sum_environment_df_2019_2020.before_mobility == 1) |\
                             (sum_environment_df_2019_2020.after_normalcy == 1)].drop(labels = ['after_mobility', 'before_normalcy'], axis = 1).copy()
sum_environment_df_2019_2020_before_mobility_after_normalcy['period'] = (sum_environment_df_2019_2020_before_mobility_after_normalcy.after_normalcy ==1).astype(int)
sum_environment_df_2019_2020_before_mobility_after_normalcy = sum_environment_df_2019_2020_before_mobility_after_normalcy[sum_environment_df_2019_2020_before_mobility_after_normalcy.language != 'ca']

# after mobility and after normalcy
sum_environment_df_2019_2020_after_mobility_after_normalcy = sum_environment_df_2019_2020[(sum_environment_df_2019_2020.after_mobility == 1) |\
                             (sum_environment_df_2019_2020.after_normalcy == 1)].drop(labels = ['before_mobility', 'before_normalcy'], axis = 1).copy()
sum_environment_df_2019_2020_after_mobility_after_normalcy['period'] = (sum_environment_df_2019_2020_after_mobility_after_normalcy.after_mobility ==1).astype(int)
sum_environment_df_2019_2020_after_mobility_after_normalcy = sum_environment_df_2019_2020_after_mobility_after_normalcy[sum_environment_df_2019_2020_after_mobility_after_normalcy.language != 'ca']

For the 3 regressions, the regression formula used is the same: $log\_views \; \text{~}\;is\_2020 * period * language  $, where :
- is_2020 is a binary variable that determines whether it is 2019 or 2020
- period is a binary variable that determines whether we're in a certain time period for each test (for example, it is equal to 1 in the first regression if the data point is in the 5 week window after the mobility for the corresponding language, otherwise it is before). When both this variable is equal to 1, and the year is 2020, then we are in the treated group.
- language simply corresponds to that data point's language; lets the model adapt to the different languages pageview volumes

#### Difference in difference

In [ ]:
all_languages = sum_environment_df_2019_2020_before_mobility_after_normalcy.language.unique()

In [ ]:
def get_standard_error_sum(results, covariates):
    '''
    #95CI is approximated with +- 2 sum_variance_standard_error
    '''
    # get the variance covariance matrix
    # print(covariates)
    vcov = results.cov_params() \
        .loc[covariates, covariates].values

    # calculate the sum of all pair wise covariances by summing up off-diagonal entries
    off_dia_sum = np.sum(vcov)
    # variance of a sum of variables is the square root
    return np.sqrt(off_dia_sum)

In [ ]:
def diff_n_diff(df, careful_da = False):
    mod = smf.ols(formula="""log_pages_views ~ is_2020* period * C(language) """ , data=df)
    res = mod.fit()
    #print(res.params)
    res = res.get_robustcov_results(cov_type='HC0')
    res = statsmodels.regression.linear_model.RegressionResultsWrapper(res)
    print("R2 for this regression: {}".format(res.rsquared))
    df_list = []
    for lang in all_languages:
        if careful_da and lang == 'da':
            val = res.params['is_2020:period']

            std = get_standard_error_sum(res, ['is_2020:period'])
        else:
            val = res.params['is_2020:period'] + \
                    res.params['is_2020:period:C(language)[T.{}]'.format(lang)]

            std = get_standard_error_sum(res, ['is_2020:period',
                                               'is_2020:period:C(language)[T.{}]'.format(lang)])
        tmp_dict = {
                "lang": lang,
                "low": val - 2 * std,
                "high": val + 2 * std,
                "val": val,
                "pval": (val - 2 * std > 0) or (val + 2 * std < 0),
                "std": std
            }

        df_list.append(tmp_dict)
    print("==============================================")
    print(pd.DataFrame(df_list))

We will now have a look at the results from the 3 different regressions separately.

In [ ]:
# Pre-vs-post mobility change effects on environment views
diff_n_diff(sum_environment_df_2019_2020_before_after_mobility)

Let us now try to interpret the results we have for the first regression:
- First, notice that for the logarithmic pre-vs.-post mobility effect isn't significant for all languages; only the Japanese, Italian, Serbian, Dutch and Finnish observe a statistically meaningful change. We also notice that for the languages where the effect isn't significant, the coefficients either go in the negatives (i.e there is a diminution of environment views after the change of mobility), or in very small positive values (i.e, small multiplicative effect).
- Second, for every statistically significant change, there is an increase that can go from around 116% of normal pageviews for the Japanese Wikipedia ($exp(val)\; \text{≃} 1.16$ ), to around 224 % for the Serbian one. 

In [ ]:
# Pre mobility vs Post normalcy effects on environment views
diff_n_diff(sum_environment_df_2019_2020_before_mobility_after_normalcy, True)

For this second regression, we notice that:
- for a majority of languages, the change is statistically significant, i.e there is indeed a difference between how people visit environment related pages before the mobility changes, and after the return to normalcy.

- for most of these meaningful changes, we see a decrease, i.e people visit these pages less after returning to normal mobility. There are some outliers for which there is an increase (Serbian and Japanese), but for the decreases can range from 87% (Italian) of before-mobility pageviews to 74% (Danish), which is the most significant. 

In [ ]:
# Post mobility vs Post normalcy effects on environment views
diff_n_diff(sum_environment_df_2019_2020_after_mobility_after_normalcy, True)

For the final results, this is what we have:
- First, for all statistically significant changes, we observe an increase after the mobility changes compared to the post-return to normalcy window. We also see that whereas the change for some languages wasn't significant in the pre-vs-post mobility regression, it became so in this one (Danish and French for example). 

- Relating these results to the two above regressions, we see that it makes sense: according to the first analysis, views generally increased post-mobility compared to the pre-mobility, and according to the second, views generally decreased after the return to normality compared to before the mobility change. Therefore by transitivity is it expected that on average, there would be even bigger changes when comparing post-mobility to post-normalcy.

After this analysis, we can conclude that depending on the considered time period and year, a general increase of environment views is clear after the mobility change compared to both after a return to normalcy, and before that same mobility change for every language.

Now, this doesn't mean that people are necessarily in the topic of environment itself; as such, the next step of our analysis will be to compare the evolution of these views with the whole of Wikipedia for every language.

## Comparaison with rest of Wikipedia

First, we will start with a simple plot, to gather some first-order intuition: we will aggregate, for every language, all the views from all topics one hand, and the environment views on the other. We will then plot the trends using a period of 14 days and see if the overall evolutions are the same.

In [ ]:
sum_environment_df['year'] = sum_environment_df.date.dt.year
sum_environment_df_2020 = sum_environment_df[sum_environment_df['year'] > 2019][['environment_views', 'language', 'date']]
sum_environment_df_2020_views = sum_environment_df_2020.groupby('date').sum()
sum_data_df_all_2020_views = sum_data_df[sum_data_df.date.dt.year > 2019].groupby('date').sum()
#sum_data_df_all_2020_views.head()

In [ ]:
result_env = seasonal_decompose(sum_environment_df_2020_views, model='additive', period=14).trend
result_all = seasonal_decompose(sum_data_df_all_2020_views, model='additive', period=14).trend
# To put them both on the same scale from 0 to 1, we normalize by the maximum of every trend
result_env = result_env / result_env.max()
result_all = result_all/ result_all.max()

In [ ]:
figure(figsize=(8, 6), dpi=80)
plt.plot(result_env, label = "Environment views")
plt.plot(result_all, label = "All views")
plt.legend()

As we can see in this first analysis, it appears that the maximum of the environment views happen in the beginning of 2020, during a time where the Wikipedia views in general are declining; after that, in the first part of March, the general views are in constant increase while the environment ones decline until near the end of March. After that, both timeseries appear to be in sync. 

In order to be more fine-grained about our analysis, we will propose two methods : Dynamic Time Warping first, then Time Lagged Cross-Correlation.

### Dynamic time warping

Dynamic time warping (DTW) is a way to compare two usually temporal sequences that do not sync up perfectly. It is a method to calculate the optimal matching between two sequences. It’s commonly used to measure the distance between two time-series.

Our goal here is to build some intuition on whenever the increase in popularity of the environement pages is simply due to the fact that people read wikipedia more or if people are really more interested in them. In order to do that we will study the similarity between the wikipedia views of environement related topics with the total wikipedia views both as a function of time. We only keep the views data from 2020.

In [ ]:
sum_data_df_2020=sum_data_df[sum_data_df['date']>='2020-01-01']
sum_environment_df_2020=sum_environment_df[sum_environment_df['date']>='2020-01-01']
language=set(sum_data_df_2020['language'])

In [ ]:
i = 0
fig, axs = plt.subplots(7, 2)
fig.set_figheight(35)
fig.set_figwidth(15)

for code in language:
    ax = axs[i % 7, i // 7 ]
    """
     We decided to use min max scaling as it makes the distance easier to interpret.
     Scaling is important as both timeseries don't have the same range of values, and so this way both timeseries are in the [0, 1] range which makes 
     it possible to compare them.
    """
    sum_data_df_2020_normalized = sum_data_df_2020[(sum_data_df_2020['views'].notnull()) & (sum_data_df_2020['language']==code)].copy()
    sum_data_df_2020_normalized['views'] = (sum_data_df_2020_normalized['views']-sum_data_df_2020_normalized['views'].min())/sum_data_df_2020_normalized['views'].max()
    
    sum_environment_df_2020_normalized = sum_environment_df_2020[sum_environment_df_2020['environment_views'].notnull()& (sum_data_df_2020['language']==code)].copy()
    sum_environment_df_2020_normalized['environment_views']=(sum_environment_df_2020_normalized['environment_views']-sum_environment_df_2020_normalized['environment_views'].min())/sum_environment_df_2020_normalized['environment_views'].max()
    
    d, cost_matrix, acc_cost_matrix, path = accelerated_dtw(sum_data_df_2020_normalized['views'].values,sum_environment_df_2020_normalized['environment_views'].values,dist='euclidean')

    ax.imshow(acc_cost_matrix.T, origin='lower', cmap='plasma', interpolation='nearest')
    ax.plot(path[0], path[1], 'w')
    ax.set_xlabel('global views '+code)
    ax.set_ylabel('environment_views '+code)
    ax.set_title(f'DTW Minimum Path with minimum distance: {np.round(d,2)}')
    i+=1

We can see here that for Italian, Norwegian, English and Dutch, the time series are really close to each others, and because the shortest path is really close to the matrix diagonal, we can say that the time series are behaving similarly (up to the scales of the values and time dilation). This may indicate that the evolution in environment views isn't due to a sudden ecological awareness.

For some of the other languages (Korean or Swedish for example), the distances on one hand are higher than average, and the plots are very far from being lines. This means that DTW didn't find a mapping that is even close to one-to-one for a majority of the points; i.e, the environment views and the total Wikipedia evolution for these languages are different.

### Time lagged cross correlation

Another similarity study we decided to use as a complement to the DTW is the Time lagged cross correlation which study the behavior of $\mathbf{corr}[X_tY_{t+\tau}]$ where $\tau$ is a time lag. The goal of this analysis is see for which value of $\tau$ the correlation is maximal in order to determine if one series is a shifted version of the other or at least has a non zero correlation of time lagged version of the other.

In [ ]:
def crosscorr(datax, datay, lag=0, wrap=False):
    """ Lag-N cross correlation. 
    Shifted data filled with NaNs 
    
    Parameters
    ----------
    lag : int, default 0
    datax, datay : pandas.Series objects of equal length
    Returns
    ----------
    crosscorr : float
    """
    if wrap:
        shiftedy = datay.shift(lag)
        shiftedy.iloc[:lag] = datay.iloc[-lag:].values
        return datax.corr(shiftedy)
    else: 
        return datax.corr(datay.shift(lag))
for code in language:
    sum_data_df_2020_normalized = sum_data_df_2020[(sum_data_df_2020['views'].notnull()) & (sum_data_df_2020['language']==code)].copy()
    sum_data_df_2020_normalized['views'] = (sum_data_df_2020_normalized['views']-sum_data_df_2020_normalized['views'].min())/sum_data_df_2020_normalized['views'].max()

    sum_environment_df_2020_normalized = sum_environment_df_2020[sum_environment_df_2020['environment_views'].notnull()& (sum_data_df_2020['language']==code)].copy()
    sum_environment_df_2020_normalized['environment_views']=(sum_environment_df_2020_normalized['environment_views']-sum_environment_df_2020_normalized['environment_views'].min())/sum_environment_df_2020_normalized['environment_views'].max()



    d1 = sum_data_df_2020_normalized['views']
    d2 = sum_environment_df_2020_normalized['environment_views']
    seconds = 5
    fps = 30
    rs = [crosscorr(d1,d2, lag) for lag in range(-int(seconds*fps),int(seconds*fps+1))]
    offset = np.floor(len(rs)/2)-np.argmax(rs)
    f,ax=plt.subplots(figsize=(14,3))
    ax.plot(rs)
    ax.axvline(np.ceil(len(rs)/2),color='k',linestyle='--',label='Center')
    ax.axvline(np.argmax(rs),color='r',linestyle='--',label='Peak synchrony')
    ax.set(title=f'Offset = {offset} frames\nS1 leads <> S2 leads for '+code,ylim=[-1,1],xlim=[0,301], xlabel='Offset',ylabel='Pearson r')
    ax.set_xticks([0, 50, 100, 151, 201, 251, 301])
    ax.grid()
    ax.set_xticklabels([-150, -100, -50, 0, 50, 100, 150]);
    plt.legend()

An interesting case here is German for which an offset of 140 has a Pearson coefficient of 0.75 meaning that the wikipedia views at time $t+140$ almost overlaps with environment views at time t. This might indicate that some increase in the views of the environement related topics might not be due to an increase in wikipedia views.
Meanwhile, for Serbian, Catalan, French, Italian and Norwegian, the offset that maximizes the cross correlation is 0. As such, we cannot really conclude much as the wikipedia views and environement views behave very similarly, so any increase in the environement views appears to be due to just an increase in Wikipedia views in general.

### Study of the evolution of the rank of the environement topic as a function of time

The previous analysis showed that a more in depth study of the Wikipedia views is required to truly see if the people's attention shifted towards or away from the environment pages during the first Covid wave. To that end, we decided to study the ranking of the views of the environment compared to the rest, to see whether or not it is one of the most important topics. A ranking of 1 means that it is the most viewed subject, while a ranking of 64 means the opposite.

In [ ]:
topics = set(countries_to_topics_sum_df.columns)
df_per_topic=dict()

for topic in topics:
    sum_topic_df = countries_to_topics_sum_df[topic]
    country_to_topic_data_sum = {}
    for country in sum_topic_df.index:
        country_to_topic_data_sum[country] = sum_topic_df[country]
    sum_topic_df = pd.DataFrame.from_dict(country_to_topic_data_sum, orient="index").T

    new_sum_topic_df = pd.DataFrame()
    for language in languages_unique:
        country_topic_sum_data = pd.DataFrame()

        country_topic_sum_data[topic+"_views"] = (
            sum_topic_df[language] + sum_topic_df[language + ".m"]
        )
        country_topic_sum_data["language"] = language
        new_sum_topic_df = pd.concat(
            [new_sum_topic_df, country_topic_sum_data], axis=0
        )
    sum_topic_df = new_sum_topic_df
    sum_topic_df.index = pd.to_datetime(sum_topic_df.index)
    sum_topic_df["date"] = sum_topic_df.index
    sum_topic_df["topic"] = topic
    sum_topic_df = sum_topic_df[sum_topic_df.date.dt.year > 2018]
    sum_topic_df = sum_topic_df.rename(columns = {topic+'_views':'views'})
    df_per_topic.update([(topic,sum_topic_df)])
multi_language_save_topic_data = copy.deepcopy(df_per_topic)
    
# we sum the views of all languages together

for topic in topics:
    df_per_topic[topic]=df_per_topic[topic].groupby('date').sum()
    
for topic in topics:
    df_per_topic[topic]['topic']=topic
    df_per_topic[topic].rename(columns = {topic+'_views':'views'}, inplace = True)

df = pd.DataFrame()
for topic in topics:
    df = pd.concat([df, df_per_topic[topic]])   
df['date']=df.index   
df['Month_Year'] = df['date'].dt.to_period('M')
df2=pd.DataFrame(df.groupby(['topic','Month_Year'])['views'].sum())
periods=set(df['Month_Year'])
df2['Month_Year']=df2.index.get_level_values('Month_Year')
df2['topic']=df2.index.get_level_values('topic')
environement_rank=pd.DataFrame()
for period in periods:
    temp=df2[df2['Month_Year']==period].copy()
    temp['topic']=temp.index.get_level_values('topic')
    temp['rank']=temp['views'].rank(ascending=False)
    temp[temp['topic']=='STEM.Earth and environment']['rank']
    environement_rank=pd.concat([environement_rank,temp[temp['topic']=='STEM.Earth and environment']['rank']])
environement_rank.index=environement_rank.index.map( lambda  x:x[1])
environement_rank=environement_rank.sort_index(ascending=True)

In [ ]:
environement_rank.plot.line(figsize = (16,5))
plt.title('Evolution of the ranking of the views of the environement topic per month')
plt.xlabel('Date')
plt.ylabel('Environment rank')
plt.legend(["Rank"])
plt.show()

The results here aren't the best; the fact is that the rank of the environment topic (aggregated accross all available languages) doesn't go above 44, and towards June/July, it actually goes above the 50 rank threshold.

To try and see if this is the result of some languages dominating others in terms of views (English for example), and thus shifting the ranks, we will do the same ranking plot but for every language.

In [ ]:
language_to_env_ranking = {} #will hold the ranking of env for each languages
periods = set()
for language in languages_unique:
    df_language_all_topics = pd.DataFrame()
    for topic in topics:
        language_topic_data =  multi_language_save_topic_data[topic]
        language_topic_data = language_topic_data[language_topic_data.language == language]
        df_language_all_topics = pd.concat([df_language_all_topics, language_topic_data])
    #print(df_language_all_topics)
    df_language_all_topics['year_month'] = df_language_all_topics['date'].dt.to_period('M')
    if(periods == set()):
        periods = sorted(set(df_language_all_topics['year_month']))
    
    grouped = pd.DataFrame(df_language_all_topics.groupby(['year_month', 'topic'])['views'].sum())
    grouped['year_month'] = grouped.index.get_level_values('year_month')
    grouped['topic'] = grouped.index.get_level_values('topic')
    #print(grouped.year_month)
    #grouped
    result = []
    result_language = pd.DataFrame()
    for period in periods:
        tmp = grouped[grouped.year_month == period].copy()
        tmp['rank'] = tmp['views'].rank(ascending = False)
        tmp = pd.concat([result_language, tmp[tmp.topic == 'STEM.Earth and environment'][['year_month', 'rank']]])
        tmp.index = tmp.year_month
        result_language = tmp
        #result.append(tmp[tmp.topic == 'STEM.Earth and environment']['rank'])
    language_to_env_ranking[language] = result_language
    

In [ ]:
plt.figure(figsize = (16,5))
plt.gca().set_prop_cycle(None)
cmap = plt.get_cmap('jet_r')
for idx,language in enumerate(languages_unique):
    color = cmap(float(idx)/len(languages_unique))
    language_to_env_ranking[language] = language_to_env_ranking[language].rename(columns = {'rank': language + "_rank"})
    ax = language_to_env_ranking[language][language + "_rank"].plot.line(color = color)
plt.title('Evolution of the ranking of the views of the environement topic per month and per language')
plt.xlabel('Date')
plt.ylabel('Rank')
plt.legend()
plt.show()

Here, the results aren't looking great either: it appears that a majority of the languages, the didn't really change their focus towards environment when the first Covid wave hit in March/April 2020. One language in particular that appears to have that behavior is Serbian, but even that shift may have not been caused by Covid; other changes seem to be seasonal.

## Conclusion

Excluding the data loading/wrangling, this first part serves as a preliminary analysis of the Coronawiki dataset content related to environmental topics. Firstly, we saw that statistically speaking, for a majority of the available countries, there was indeed a difference between the visits to environmental topics between the years of 2019 and 2020; this difference wasn't always an increase for all languages.

Using that as a basis, we asked ourselves whether we can predict these views using only the changes in transportations mode/visits to certain places, for every language. We saw that it appears to be possible, but not always interpretable, and there may be a lack of contextualisation, i.e it's not certain that if we had the same data but from another year (for example 2019), we would have the same results. In order to do that, we resort to difference-in-difference regression.

Depending on the considered time period and year, a general increase of environment views is clear after the mobility change compared to both after a return to normalcy, and before that same mobility change for every language. Note that in this case, we did use the 2019 data as well, so we see that the fact that there is an increase in 2020 is indeed related to Covid-related restrictions. However, we asked ourselves another question at this point: yes, there is an increase happening during the first Covid wave, but this translate directly into environmental awareness?

To attempt to answer that, we decided to study the similarities between the evolution of the global wikipedia views, and the evolution of wikipedia views related to the environement.
The DTW analysis showed that for a lot of languages, both of the corresponding timeseries are really close to each others in distance and behave very similarly. This seems to indicate that an increase in environment views was most of the time, because of an increase Wikipedia usage in general. 

In order to complement this analysis, we decided to look at the time lagged cross correlation. Unfortunatly, other than for German for which time lagged cross correlation attains its maximun of 0.75 with a lag of 140 compared to the unlagged correlation of 0.5, most of the languages achieve maximum cross correlation for a lag of 0 or 1, and therefore both of their time series are almost superposed.

More over, since the environement views and wikipedia views behave the same, we decided to study the ranking of the environement topic compared to other topics, either by separating the languages or aggregating them all together. The results we have seems to indicate that people were more preoccupied with other topics than the environment during the first Covid wave.

Thus it seems that for now, the increase in environement views in 2020 isn't necessarily due to a sudden awarness as its rankind drops to 51, but to a increase in wikipedia views. However, it doesn't mean that more awareness wasn't there; our data does stop in July 2020 after all; plus, the Wikipedia views alone aren't enough to confirm a change in awareness, we need to have a better look at the general behavior of people. In order to complement our analysis, we add the usage in the followning part of external datasets related to pollution, and even more Wikipedia data to see the evolution of environment related views after 2020. 

# Part 2 : The external datasets which we added by ourselves

## Precise Wikipedia Views Analysis

The goal of this section is to precisely analyse the views of different wikipedia pages (Air pollution, Plastic pollution, and plastic production) from 01/01/2019 to 01/09/2022.

## Data import

In [ ]:
# We restrict the page to the language studied in coronawiki
languages = [
    "ja",
    "it",
    "da",
    "tr",
    "no",
    "en",
    "sr",
    "sv",
    "nl",
    "de",
    "fr",
    "ca",
    "ko",
    "fi",
]

In [ ]:
def process_langviews_data(path, columns_to_keep=None):
    """
    Convert the csv generated by langviews for a particular wikipedia page into a usable dataframe keeping only the pages in the specified languages.
    The resulting dataframe has a column per language and a new total column giving the total number of views across all the studied languages.
    """

    data = pd.read_csv(path).transpose()
    data.columns = data.iloc[0]
    titles = data.iloc[1]
    data.drop(["Language", "Title", "Badges"], inplace=True)

    if columns_to_keep is None:
        columns_to_keep = data.columns

    columns = data.columns.intersection(columns_to_keep + ["date"])
    data = data[columns]

    data["total"] = data.sum(axis=1)
    data["date"] = data.index
    data["date"] = pd.to_datetime(data["date"])
    data["year"] = data.apply(lambda x: x.date.year, axis=1)

    return  titles ,data

#### Air Pollution

In [ ]:
airpol_titles, airpol_data = process_langviews_data(
    "Data/Additional/wikiviews/page-views-airpol.csv", languages
)
airpol_data

#### Plastic Industry

In [ ]:
plastin_titles, plastin_data = process_langviews_data(
    "Data/Additional/wikiviews/page-views-plastin.csv", languages
)
plastin_data

#### Plastic Pollution

In [ ]:
plastpol_titles, plastpol_data = process_langviews_data(
    "Data/Additional/wikiviews/page-views-plastpol.csv", languages
)
plastpol_data

### Covid-19

In [ ]:
covid_titles, covid_data = process_langviews_data(
    "Data/Additional/wikiviews/page-views-covid19.csv", languages
)
covid_data

### Total Wikipedia Views

In [ ]:
global_views = pd.read_csv('Data/Additional/langviews/final/wikipedia_global_views.csv')
global_views['date'] = pd.to_datetime(global_views['date'])
global_views['total'] = global_views.drop("date", axis=1).sum(axis=1)
global_views['year'] = global_views.apply(lambda x: x.date.year, axis=1)

## Visualization

### Air Pollution

In [ ]:
sns.boxplot(y=airpol_data.total, x=airpol_data.year)
plt.title("Air Pollution page views count")
plt.xlabel("Date")
plt.ylabel("Count")
plt.legend(["total"])
plt.show()

It seems that during and after COVID, interest in air pollution decreases significantly

### Plastic Industry

In [ ]:
sns.boxplot(y=plastin_data.total, x=plastin_data.year)
plt.title("Plastic Industry page views count")
plt.xlabel("Date")
plt.ylabel("Count")
plt.legend(["total"])
plt.show()

There are not enough views to conclude anything about awareness on this topic.

### Plastic Pollution

In [ ]:
sns.boxplot(y=plastpol_data.total, x=plastpol_data.year)
plt.title("Plastic Pollution page views count")
plt.xlabel("Date")
plt.ylabel("Count")
plt.legend(["total"])
plt.show()

### Total Wikipedia Views

In [ ]:
sns.boxplot(y=global_views.total, x=global_views.year)
plt.title("Total Wikipedia views count")
plt.xlabel("Date")
plt.ylabel("Count")
plt.legend(["total"])

Globally, it seems that interest in wikipedia increases during Covid and significatively decreases after the crisis.

### Environment topic expansion

The coronawiki dataset contains aggregated views per day of all articles related to different topics. Unfortunately, these values are only listed for the years 2018 to 2020. This is not usable to do a proper analysis, so we used the langviews website to acquire views data for every article in the environment topic and aggregate them. To see how we precisely did, you can see the *wikipedia_views_extension* notebook.

In [ ]:
aggregated_views = pd.read_csv('Data/Additional/langviews/final/aggregated_views.csv')
aggregated_views['date'] = pd.to_datetime(aggregated_views['date'])
aggregated_views['year'] = aggregated_views.apply(lambda x: x.date.year, axis=1)
aggregated_views

In [ ]:
sns.boxplot(y=aggregated_views.total, x=aggregated_views.year)
plt.title("Environment topic views count")
plt.xlabel("Date")
plt.ylabel("Count")
plt.legend(["total"])
plt.show()

Globally, after the Covid crisis, interest in articles concerning the environment topic decreases. We now need to determine wether this decrease follow the general loss of interest in wikipedia or not.

This concludes the extra data from Wikipedia that we add to the original Coronawiki dataset.
Let us now present the next one : NO2 air pollution data by country.

## Analysis of nitrogen dioxide levels by capital city

Nitrogen dioxide is a toxic molecule that is typically emitted by industrial activities and car engines. It is therefore a very good indicator of human-induced air pollution.

Here, we will focus on understanding the link between Covid and NO2 air pollution, i.e. between Covid and human-induced air pollution. To do so, we will make the following assumptions :
- 2019 is the year that represents the "usual activities" before Covid.
- The year of lockdowns and restrictions is then obviously 2020. We also consider that 2021-22 are the beginning of efforts towards going back to the normal, pre-Covid life. We do not assume that 2021 and 2022 have succeeded at being "back to normal".

This might be a simplification of reality, but it has the two following advantages :
- Covid began spreading worldwide during the very beginning of 2020. We can exploit this to our advantage by splitting years at the beginning of January, which is easy and fits our assumptions well.
- It makes for very easy cross-coutries, cross-years comparisons.
- Using instead lockdown data instead of year-by-year separations would weaken the possibility of comparing pollution by country, as pollution emissions are very dependent on the period of the year.

The goal of this section is then to establish (or not) the fact that during Covid, the air got significantly cleaner, and to quantify the margin of improvement. We also consider the recovery phase after the initial Covid wave, i.e. after 2020.

## Data loading

We use air pollution data from the different capitals from the World Air Quality Index dataset for the Covid-19 period (https://aqicn.org/data-platform/covid19/). This is a reliable dataset that was used in various large-scale studies about worldwide air pollution. This dataset is fairly lage, it includes the data for the years 2018 to 2022 for many cities of the world, and for various air polluting molecules.

We reduce this to NO2 and only the fourteen capital cities in the notebook "air_quality_by_capital.ipynb", reducing the dataset to a much more manageable size, but not to the point of losing all interesting information.

Note : we don't consider country-wide pollution because the dataset does not provide us with a way to merge all cities into one large blob for the country : we don't have the city size, the geographic proportion of the city in the country, etc. We therefore study capitals only, which the dataset certainly provides.

In [ ]:
path_to_datasets = "Data/Additional/waqi/no2_capital/"
capitals = [
    "tokyo",
    "rome",
    "copenhagen",
    "ankara",
    "oslo",
    "washington",
    "stockholm",
    "belgrade",
    "amsterdam",
    "berlin",
    "paris",
    "barcelona",
    "seoul",
    "helsinki",
]

In [ ]:
full_datasets = []
for capital in capitals:
    full_datasets.append((capital, pd.read_csv(path_to_datasets + capital + ".csv")))

In [ ]:
full_datasets[1][1]

In [ ]:
ds_2019 = []
for (capital, dataset) in full_datasets:
    # keep year 2019
    ds = dataset[dataset.Date.str.startswith("2019")].copy()
    # remove year from date string to allow for inner merge on date with other years later on
    ds["yearlessDate"] = ds.Date.str[5:]
    # remove all other columns
    ds = ds[["yearlessDate", "median"]]
    ds_2019.append((capital, ds))

In [ ]:
# same thing as above, but for 2020
ds_2020 = []
for (capital, dataset) in full_datasets:
    ds = dataset[dataset.Date.str.startswith("2020")].copy()
    ds["yearlessDate"] = ds.Date.str[5:]
    ds = ds[["yearlessDate", "median"]]
    ds_2020.append((capital, ds))

In [ ]:
ds_2021 = []
for (capital, dataset) in full_datasets:
    ds = dataset[dataset.Date.str.startswith("2021")].copy()
    ds["yearlessDate"] = ds.Date.str[5:]
    ds = ds[["yearlessDate", "median"]]
    ds_2021.append((capital, ds))

In [ ]:
ds_2022 = []
for (capital, dataset) in full_datasets:
    ds = dataset[dataset.Date.str.startswith("2022")].copy()
    ds["yearlessDate"] = ds.Date.str[5:]
    ds = ds[["yearlessDate", "median"]]
    ds_2022.append((capital, ds))

## Data viz

We will then plot the SO3 measurements in all 14 cities every day for the years 2019, 2020 and 2021.

We could also easily plot the current data for 2022, but we found that the resulting graph was fairly overloaded.

We use a log scale because these cities have strong differences in air quality, and we want to show the full detail.

In [ ]:
fig, ax = plt.subplots(14, sharex=True, sharey=True)
fig.set_size_inches(18.5, 35.5)
for i in range(14):
    ds1 = ds_2019[i][1]
    ds2 = ds_2020[i][1]
    ds3 = ds_2021[i][1]
    # not 2022, see above

    ax[i].set_yscale("log")
    ax[i].set_title(ds_2019[i][0])
    # always 2020 because we want all three graphs on one year, and 2020 is a leap year
    (p2019,) = ax[i].plot_date(
        matplotlib.dates.datestr2num("2020-" + ds1["yearlessDate"]),
        ds1["median"],
        tz="UTC+1",
        fmt="-",
        linewidth=0.8,
    )
    (p2020,) = ax[i].plot_date(
        matplotlib.dates.datestr2num("2020-" + ds2["yearlessDate"]),
        ds2["median"],
        tz="UTC+1",
        fmt="-",
        linewidth=0.8,
    )
    (p2021,) = ax[i].plot_date(
        matplotlib.dates.datestr2num("2020-" + ds3["yearlessDate"]),
        ds3["median"],
        tz="UTC+1",
        fmt="-",
        linewidth=0.8,
    )
    ax[i].legend([p2019, p2020, p2021], ["2019", "2020", "2021"])

Some trends emerge :
- 2019 (in blue) is usually a little bit above the others. When considering that this is a log scale, this is actually a fairly impressive difference.
- 2020 is typically lower. We can almost always find a drop in March 2020, where the international community initially reacted to the virus.
- 2020 and 2021 are somewhat more difficult to discern. It could be that these years are similar in terms of NO2 pollution.

We can also check the evolution of mean pollution per country per year :

In [ ]:
means2019 = [df["median"].mean() for (capital, df) in ds_2019]
means2020 = [df["median"].mean() for (capital, df) in ds_2020]
means2021 = [df["median"].mean() for (capital, df) in ds_2021]
means2022 = [df["median"].mean() for (capital, df) in ds_2022]


plt.title("Yearly average of daily pollution in capital cities")

for i in range(len(capitals)):
    plt.plot(
        ["2019", "2020", "2021", "2022"],
        [means2019[i], means2020[i], means2021[i], means2022[i]],
        linewidth=0.8,
        label=capitals[i],
    )

plt.legend(loc=(1.05, 0))
plt.show()

Perhaps surprisingly, the graph is not U-shaped at all ! This means that on average, air pollution in the capital cities goes down steadily year after year. There is a caveat : the year 2022 is not over as we write these words. This means that the end of autumn and beginning of winter 2022 are not accounted for in this graph. If winter is more polluted than summer in 2022, then the graph above underestimates the pollution average for 2022.

We can then quantify this evolution exactly, by using statistics.

## Statistical testing

The testing we will perform consists of the following :
- First, we merge the pollution data of two years
- Then, for each capital city, we do a paired test comparing pairs of (day, day) of both years
- The two days are always on the same date, except for the year. This enables us to compare pollution day by day, ignoring the effects of recurring seasonal pollution. This analysis works for 2022, as the missing data for the end of the year will be ignored in the other dataset, possibly making the compromise of increasing the variance of test statistics.

We first merge years we want to compare. We will compare the following pairs : (2019-2020) to check if covid had an impact against the 2019 baseline, (2020-2021) to see if 2021 was a rebounce/drop from Covid, and (2020, 2022) for the same reason.

In [ ]:
merged19_20 = []
for index in range(len(capitals)):
    (capital1, df1) = ds_2019[index]
    (capital2, df2) = ds_2020[index]
    assert capital1 == capital2
    mergeTwoYears = pd.merge(df1, df2, on="yearlessDate", how="inner")
    merged19_20.append((capitals[index], mergeTwoYears))

merged20_21 = []
for index in range(len(capitals)):
    (capital2, df2) = ds_2020[index]
    (capital3, df3) = ds_2021[index]
    assert capital2 == capital3
    mergeTwoYears = pd.merge(df2, df3, on="yearlessDate", how="inner")
    merged20_21.append((capitals[index], mergeTwoYears))

merged20_22 = []
for index in range(len(capitals)):
    (capital2, df2) = ds_2020[index]
    (capital4, df4) = ds_2022[index]
    assert capital2 == capital4
    mergeTwoYears = pd.merge(df2, df4, on="yearlessDate", how="inner")
    merged20_22.append((capitals[index], mergeTwoYears))

We can then do the daily paired test for the year pairs :

2019 to 2020 comparison :

In [ ]:
test_by_capital = []
for capital, merger in merged19_20:
    ttest = stats.ttest_rel(merger.median_x, merger.median_y, alternative="greater")
    alone_sig = ttest.pvalue < 0.05
    bonf_sig = ttest.pvalue < (0.05 / 14)
    evolution = (
        "{:+.2f}".format(
            ((np.mean(merger.median_y) / np.mean(merger.median_x)) - 1) * 100
        )
        + "%"
    )

    test_by_capital.append((capital, len(merger), evolution, alone_sig, bonf_sig))

pd.DataFrame(
    test_by_capital,
    columns=[
        "Capital",
        "numpoints",
        "pollution_evolution",
        "better_during_covid_significant",
        "bonferroni_significant",
    ],
)

Here, we learn that for almost every capital city, the air was significantly cleaner in during Covid times than before. There is an exception for Ankara in Turkey, which is the only capital city that polluted more during Covid than before. Tokyo is not Bonferroni-significant, but also shows a drop in average pollution during Covid.

In [ ]:
test_by_capital = []
for capital, merger in merged20_21:
    # NOTE : we switch to two-sided testing because we are interested in both kinds of changes.
    # Before, we were only interested in knowing whether covid was an improvement or not
    ttest = stats.ttest_rel(merger.median_x, merger.median_y, alternative="two-sided")
    alone_sig = ttest.pvalue < 0.05
    bonf_sig = ttest.pvalue < (0.05 / 14)
    evolution = (
        "{:.2f}".format(
            ((np.mean(merger.median_y) / np.mean(merger.median_x)) - 1) * 100
        )
        + "%"
    )

    test_by_capital.append((capital, len(merger), evolution, alone_sig, bonf_sig))

pd.DataFrame(
    test_by_capital,
    columns=[
        "Capital",
        "numpoints",
        "pollution_evolution",
        "alone_significant",
        "bonferroni_significant",
    ],
)

Here, the results are much less significant, as the test could not manage to find a lot of cities where 2020 and 2021 were somehow different. We only have Belgrade (Serbia) and Helsinki (Finland) that showed a significant boost in pollution between 2020 and 2021. In this sense, the years 2020 and 2021 and very much alike.

In [ ]:
test_by_capital = []
for capital, merger in merged20_22:
    ttest = stats.ttest_rel(merger.median_x, merger.median_y, alternative="two-sided")
    alone_sig = ttest.pvalue < 0.05
    bonf_sig = ttest.pvalue < (0.05 / 14)
    evolution = (
        "{:.2f}".format(
            ((np.mean(merger.median_y) / np.mean(merger.median_x)) - 1) * 100
        )
        + "%"
    )

    test_by_capital.append((capital, len(merger), evolution, alone_sig, bonf_sig))

pd.DataFrame(
    test_by_capital,
    columns=[
        "Capital",
        "numpoints",
        "pollution_evolution",
        "alone_significant",
        "bonferroni_significant",
    ],
)

This is perhaps the surprise of this study. We find that between 2020 and 2022, the only significant changes show that the cities are doing better in 2022 in terms of air pollution, the only exception being Barcelona (Spain).

## Conclusion

Let us then conclude about the NO2 air pollution. We have established the following :
- Air pollution typically goes down with time in the fourteen capital cities we studied.
- The year 2020 is special, as it shows a massive drop worldwide. We attribute this to Covid.
- The air pollution typically does not go back up after Covid. This is surprising, as one might expect air pollution to go back up after the lockdowns are finished. This is not the case.

There are a few limitations in this study :
- We do not study pollution by country, but by capital city. If we want to draw nationwide conclusions, we need to assume that the pollution of the capital city is a good proxy for the country. This may or may not be the case.
- We assumed that we could split the timeline (before, during, after) as (2019, 2020, 2021+). This is a decent compromise in order to be able to compare the evolution country-wise, but it can be a bit rough.

This concludes this study about air pollution.

## Plastic pollution

In [ ]:
global_plastic = pd.read_csv("Data/Additional/plastic/global-plastics-production.csv")
# source : https://ourworldindata.org/plastic-pollution
global_plastic = global_plastic.rename(
    columns={"Global plastics production": "Global_plastics_production"}
)
global_plastic

In [ ]:
oecd = pd.read_excel("Data/Additional/plastic/oecd_source.xlsx")
# source: https://www.oecd.org/newsroom/plastic-pollution-is-growing-relentlessly-as-waste-management-and-recycling-fall-short.htm
oecd = oecd.dropna().rename(
    columns={
        "Unnamed: 0": "Region",
        "Unnamed: 1": "2020",
        "Unnamed: 2": "2019",
        "Unnamed: 3": "pre-COVID 2020 projection",
        "Unnamed: 4": "Absolute 2020 versus 2019",
    }
)
oecd = oecd.iloc[1:, :]

## Looking at the difference between the amount of plastic produced in 2020 and 2019 per region

In [ ]:
sns.barplot(data=oecd, x="Absolute 2020 versus 2019", y="Region").set(title="difference between the amount of plastic produced in 2020 and 2019 per region")

In [ ]:
# we dont have 2020 data here
sns.lineplot(data=global_plastic, x="Year", y="Global_plastics_production").set(title="global plastic production per year from 1950 to 2019")

Now we want to fit a regression model in order to predict what will be the global plastic production in 2020 given the data from 1950 to 2019, the goal is to see how much higher that number would have been if covid was not there 

In [ ]:
mod = smf.ols(
    formula="Global_plastics_production ~ +np.exp(Year/1950) +Year ",
    data=global_plastic,
)
model = mod.fit()
print(model.summary())

In [ ]:
X = global_plastic["Year"]
plt.plot(X, model.predict(X), color="k", label="Regression model")

plt.xlabel('year') 
plt.ylabel('model prediction') 
plt.title('The regression model') 

In [ ]:
data = dict(Year=2020)

d2020 = pd.DataFrame(data, index=[0])
plastic_prediction_2020_if_not_covid = model.predict(d2020)
real_plastic_prodction_2020=oecd['2020'][49]*10**6

In [ ]:
poucentage_de_reduction=((real_plastic_prodction_2020 - plastic_prediction_2020_if_not_covid) / plastic_prediction_2020_if_not_covid) * 100
print(poucentage_de_reduction[0])

We can see that compared to our prediction the amount of plastic produced in 2020 is 0.06% smaller

## Studying the plastic production in the EU
In the first part we study the plastic produced for food packaging, Then the whole amount of plastic produced 

In [ ]:
eu_plastic_packaging = pd.read_csv(
    "Data/Additional/plastic/sts_inpr_m__custom_3782744_linear.csv"
)
# source= https://ec.europa.eu/eurostat/databrowser/view/STS_INPR_M/default/table?lang=en production of plastic packages in europe

In [ ]:
eu_nace_r2_c222 = eu_plastic_packaging[eu_plastic_packaging["nace_r2"] == "C2222"].sort_values("TIME_PERIOD")

for region in set(eu_nace_r2_c222["geo"]):
    df = eu_nace_r2_c222[eu_nace_r2_c222["geo"] == region]
    if len(df["OBS_VALUE"].dropna()) == 0:
        continue

    df.plot.line(
        x="TIME_PERIOD", y="OBS_VALUE", title="plastic produced for packaging in " +region, rot=0
    )

Notice the drop around 2020

In [ ]:
new = eu_plastic_packaging[
    (eu_plastic_packaging["nace_r2"] == "C2222")
    & (eu_plastic_packaging["TIME_PERIOD"] >= "2020-01")
]
old = eu_plastic_packaging[
    (eu_plastic_packaging["nace_r2"] == "C2222")
    & (eu_plastic_packaging["TIME_PERIOD"] < "2020-01")
]

for each region we compare the production before and after covid to see how the pandemic affected the production

In [ ]:
for region in set(
    eu_plastic_packaging[eu_plastic_packaging["nace_r2"] == "C2222"]["geo"]
):
    if region=='IE' or region=='UK' or region=='EU28': # we have no data for these 
        continue
    print(
        "difference in the mean ammount of plastic produced before and after covid in "+region,
        stats.ttest_ind(
            new[new["geo"] == region]["OBS_VALUE"].dropna(),
            old[old["geo"] == region]["OBS_VALUE"].dropna(),
        ),
    )

it seems for France and Germany the data shows a significant drop in production, while for spain Greece and turkey we see an increase

In [ ]:
# Manufacture of plastics products by month by country
full_plastic_production = pd.read_csv(
    "Data/Additional/plastic/sts_inpr_m__custom_3857183_linear.csv"
)
# source: https://ec.europa.eu/eurostat/databrowser/view/STS_INPR_M__custom_3857183/default/table?lang=en

In [ ]:
full_plastic_production = full_plastic_production[
    full_plastic_production["TIME_PERIOD"] >= "2018-01"
]

In [ ]:
r2_c222 = full_plastic_production[full_plastic_production["nace_r2"] == "C222"].sort_values("TIME_PERIOD")
for region in set(r2_c222["geo"]):
    df = r2_c222[r2_c222["geo"] == region]
    if len(df["OBS_VALUE"].dropna()) == 0:
        continue

    df.plot.line(x="TIME_PERIOD", y="OBS_VALUE", title="plastic produced in " +region, rot=0)

notice how the drop during 2020 usually wider than the others and slightly deeper

## Study of the recycling rates and recovery rates
The goal here is to see how the recycling and recovery rates were affected by covid, since if we see a drop in plastic production it would be hard to state that it would lead to a drop in plastic pollution without studying first how covid impacted plastic management facilities.

source: https://ec.europa.eu/eurostat/databrowser/view/TEN00063__custom_3793752/default/table?lang=en

Recycling rates for packaging waste

In [ ]:
Recycling_rates_eu = pd.read_csv(
    "Data/Additional/plastic/ten00063__custom_3793752_linear.csv"
)

In [ ]:
# we filter for plastic waste
Recycling_rates_eu = Recycling_rates_eu[Recycling_rates_eu["waste"] == "W150102"]

In [ ]:
stats.ttest_ind(
    Recycling_rates_eu[Recycling_rates_eu["TIME_PERIOD"] == 2020]["OBS_VALUE"],
    Recycling_rates_eu[Recycling_rates_eu["TIME_PERIOD"] == 2019]["OBS_VALUE"],
)

The data shows that the recycling rates are unaffected by covid, thus we can conclude that the recycling rates did not change from 2019 to 2020

Rate of recovery or incineration at waste incineration plants with energy recovery’ for the purposes of Article 6(1) of Directive 94/62/EC means the total quantity of packaging waste recovered or incinerated at waste incineration plants with energy recovery, divided by the total quantity of generated packaging waste https://ec.europa.eu/eurostat/databrowser/view/ten00062/default/table?lang=en

In [ ]:
recovery_rates_eu = pd.read_csv("Data/Additional/plastic/ten00062_linear.csv")
recovery_rates_eu = recovery_rates_eu[recovery_rates_eu["waste"] == "W150102"]

In [ ]:
stats.ttest_ind(
    recovery_rates_eu[recovery_rates_eu["TIME_PERIOD"] == 2020]["OBS_VALUE"],
    recovery_rates_eu[recovery_rates_eu["TIME_PERIOD"] == 2019]["OBS_VALUE"],
)

The data shows that the recovery rates are unaffected by covid, thus we can conclude that the recovery rates did not change from 2019 to 2020

## Study of the waste generated by households

In [ ]:
eu_waste = pd.read_csv("Data/Additional/plastic/ten00110_linear.csv")
# https://ec.europa.eu/eurostat/databrowser/view/TEN00110/default/table?lang=en&category=env.env_was.env_wasgt

In [ ]:
eu_waste = eu_waste[eu_waste["waste"] == "TOTAL"]

In [ ]:
eu_waste[eu_waste["TIME_PERIOD"] == 2020]

In [ ]:
sns.lineplot(data=eu_waste, x="TIME_PERIOD", y="OBS_VALUE").set(title="Household waste generated per year in the EU")

We can see a drop of 0.5 in 2020, however, due to the low amount of data the confidence intervals are quite large, so it would be hard to perform statistical tests and get a meaningful result

## Study of the total amount of waste generated by households and businesses

In [ ]:
eu_waste = pd.read_csv("Data/Additional/plastic/ten00108_linear.csv")
eu_waste = eu_waste[eu_waste["waste"] == "TOTAL"]
# https://ec.europa.eu/eurostat/databrowser/view/TEN00108/default/table?lang=en&category=env.env_was.env_wasgt

In [ ]:
sns.lineplot(data=eu_waste, x="TIME_PERIOD", y="OBS_VALUE").set(title="Total amount of waste generated by the EU and buisnesses")

We can see a drop of almost 1 in 2020, however, due to the low amount of data the confidence intervals are quite large, so it would be hard to perform statistical tests and get a meaningful result

## Conclusion:
We saw that in some countries the plastic production experienced a significant decrease during covid, especially around 2020 we saw quite a drop probably due to lockdown. However, recycling rates and recovery rates did not experience significant change, More over The amount of waste generated decreased between 2018 and 2020 although one might question the validity of this due to a low amount of data.

Therefore with those results one might conclude that since less plastic was produced and with no impact on the recycling and recovery rates this might suggest that covid caused a drop plastic polution in the EU.


This analysis can be improved since one main issue is that the EU data lacks the data for some countries like Switzerland Poland and so on.
This causes some confidence intervals to inflate and thus makes some hypothesis hard to test.
We mainly focused on the EU and therefore we can improve it by adding other regions of the world.



# Part 3 : Studying the awareness versus the actual pollution

The goal is to establish whether there is a link between awareness (i.e. Wikipedia views) and actual ground measurements about pollution. We will perform two experiments for each country : 
- **intervention analysis** : we find the peak of wikipedia views for a given wikipedia subject or page in 2020 (the peak of awareness, which we call the intervention) and check whether this peak translates to a significant change in empirical pollution. 
- **granger causality testing** : we test whether a given timeseries (wikipedia views) can be used to linearly predict the future of another timeseries (say, pollution). This gives us a hint about the temporal relationship between two observations.

In [ ]:
country_codes = sum_environment_df.language.unique()

In [ ]:
capitals

In [ ]:
wikipedia_language_to_dataset = {
    'ja' : full_datasets[0], 
    'it' : full_datasets[1], 
    'da' : full_datasets[2],
    'tr' : full_datasets[3], 
    'no' : full_datasets[4], 
    'en' : full_datasets[5], 
    'sr' : full_datasets[7], #careful : we swap indices here 
    'sv' : full_datasets[6], 
    'nl' : full_datasets[8], 
    'de' : full_datasets[9], 
    'fr' : full_datasets[10], 
    'ca' : full_datasets[11], 
    'ko' : full_datasets[12], 
    'fi' : full_datasets[13]
}

In [ ]:
{(k, v[0]) for (k, v) in wikipedia_language_to_dataset.items()}

First, we will perform intervention analysis. The first thing we need is to find the peaks of awareness in all fourteen countries in the environment dataset.

In [ ]:
peak2020 = {}
for code in country_codes:
    datapoints = sum_environment_df[sum_environment_df.language == code]
    datapoints = datapoints[datapoints.date.dt.year == 2020]
    peak = datapoints.environment_views.max()
    peakline = datapoints[datapoints.environment_views == peak].iloc[0]
    peak2020[code] = peakline['date']

In [ ]:
peak2020

Now that we have all peaks, we can look at the 365 previous days and the 365 following years in air pollution and analyze whether there is a significant difference in air quality. This will give us an idea of the relationship between awareness of environment problems and the actual state of the environment.

#### Can wikipedia awareness be used to model an intervention in the air pollution ?

At first, let us visualize the data which we analyze : we want to see whether there is a significant difference of air quality before and after the wikipedia views peak of 2020. We will look at both the full timeseries with a change of color at the date of the peak, along with a boxplot to show the difference in quartile-based statistics.

In [ ]:
results = []
for code in country_codes:
    dataset = wikipedia_language_to_dataset[code][1]
    dataset["dt"] = pd.to_datetime(dataset.Date)
    dataset["yearless"] = dataset.Date.str[5:]
    peak = peak2020[code]
    
    before = dataset[(dataset["dt"] >= peak - datetime.timedelta(weeks=52)) & (dataset["dt"] < peak)][['Date', 'median']]
    after  = dataset[(dataset["dt"] < peak + datetime.timedelta(weeks=52)) & (dataset["dt"] >= peak)][['Date', 'median']]
    
    fig, (ax1, ax2) = plt.subplots(1, 2)
    fig.suptitle(code)
    fig.set_size_inches(13, 5)
    ax1.plot_date(matplotlib.dates.datestr2num(before["Date"]), before['median'], tz="UTC+1", fmt="-")
    ax1.plot_date(matplotlib.dates.datestr2num(after["Date"]), after['median'], tz="UTC+1", fmt="-")
    
    for tick in ax1.get_xticklabels():
        tick.set_rotation(45)
    ax2.boxplot([before["median"], after["median"]])
    plt.show()


Here's a few interesting results by themselves : 
- First of all, most countries visibly have less air pollution after the peak of their environment Wikipedia page of 2020. We will analyze this further in the next cell. That alone is however not enough to explain the whole story : both periods of 365 days have strong similarities
- Analyzing in that direction, there seems to be a group of countries that stand out : Japan, Italy, Turkey, Norway, the U.S., Serbia, the Netherlands, Norway, Korea and Finland all have a air pollution that is U-shaped, meaning they are highly seasonal and binary (summer = no pollution, winter = strong pollution). France and Germany also show a similar behavior, but with a seemingly higher variance. This is hardly the case for the last two : Denmark and Sweden are less well-behaved in terms a pollution seasonality. Looking at it globally, we can conclude that there is a strong seasonality of pollution wordlwide.

Then, we can dig into the significance of these results : is it generally true that the peak of awareness marks a significant difference in air quality in a given country ? 

In [ ]:
results = []
for code in country_codes:
    dataset = wikipedia_language_to_dataset[code][1]
    dataset["dt"] = pd.to_datetime(dataset.Date)
    dataset["yearless"] = dataset.Date.str[5:]
    peak = peak2020[code]
    
    before = dataset[(dataset["dt"] >= peak - datetime.timedelta(weeks=52)) & (dataset["dt"] < peak)][['yearless', 'median']]
    after  = dataset[(dataset["dt"] < peak + datetime.timedelta(weeks=52)) & (dataset["dt"] >= peak)][['yearless', 'median']]
    
    yearComp = before.merge(after, on='yearless')
    pval = stats.ttest_rel(yearComp['median_x'], yearComp['median_y']).pvalue
    evolution = (
        "{:+.2f}".format(
            ((np.mean(yearComp.median_y) / np.mean(yearComp.median_x)) - 1) * 100
        )
        + "%"
    )
    sig_alone = pval < 0.05
    sig_bonf = pval < 0.05/14
    results.append((code, pval, evolution, sig_alone, sig_bonf))
    
pd.DataFrame(results, columns = ['country', 'pvalue', 'evolution', 'sig_alone', 'sig_bonferonni'])

These results are in line with the rest of the air pollution analysis earlier : countries significantly reduced their air pollution between these two periods. The only exception is again Turkey. This [Wikipedia page](https://en.wikipedia.org/wiki/Air_pollution_in_Turkey#Nitrogen_oxides) gives a bit of an explanation : NOx car pollution and lack of pollution regulation are a large part of the problem. We however could not explain why the pollution goes up instead of stagnating, for example. Serbia also shows a slight increase in air pollution, but this is not a significant change from before the peak.

#### What can we conclude from this analysis of view peak vs. air pollution ?

We can establish that for the capitals of most countries studied here, **air pollution can reasonably be used as an intervention in air pollution**. However, this is not a causal analysis : we cannot really claim that awareness made the pollution go down, because of two related remarks. First, there is a strong confounder : Covid probably made people read the wikipedia page about air pollution, and at the same time made them pollute less. Second, we cannot establish causality in such sequential data, as we would need to also witness an alternative universe where covid did not happen. It is however not out of the question that awareness causes a drop in pollution : being locked down during Covid, people may have had more time to learn and think about their environment.

Bad news aside, we did learn some facts : air pollution gets significantly better after people are most aware about the problem of air pollution in their respective country. 

### Daily views vs actual pollution

We can also ask a related but different question : is it true that the value of wikipedia views of the environment topic can linearly predict the air pollution ? 

Here, the Granger test can help us. The [Granger causality test](https://en.wikipedia.org/wiki/Granger_causality) is a hypothesis testing tool that takes two time-series as input and checks whether the first one is a good linear predictor for the values of the next. It enables us to do some lag analysis, for example verifying whether the past of the first time series can predict the value of the other at a future day. Here's the implementation we will use :

In [ ]:
grangercausalitytests?

Since we are concerned with data that goes beyond what the original Coronawiki dataset gives us, we can load the extended version that goes further than 2020.

We loaded all wikipedia views on the environment topic from 2015-07 to the end of 2022. This extends the data in both directions, giving us more datapoints to merge with the air quality.

In [ ]:
sum_environment_extended = pd.read_csv("Data/Additional/langviews/final/aggregated_views.csv")
sum_environment_extended.date = pd.to_datetime(sum_environment_extended.date)

In [ ]:
sum_environment_extended

In [ ]:
res = []
for code in country_codes:
    
    env = sum_environment_extended[[code, 'date']]
    air = wikipedia_language_to_dataset[code][1]
    merged = env.merge(air, left_on="date", right_on="dt")
    
    print(code)
                                                           #jesus the access patterns are annoying
    pval = grangercausalitytests(merged[['median', code]], [1])[1][0]['ssr_chi2test'][1]
    alone_sig = pval < 0.05
    bonf_sig = pval < 0.05/14
    
    res.append((code, pval, alone_sig, bonf_sig))
    
    print("\n\n===================\n")
    


In [ ]:
air_poll_granger = pd.DataFrame(res)
air_poll_granger.columns = ["code", "pvalue", "alone_sig", "bonf_sig"]


In [ ]:
air_poll_granger

The p-values are very binary : either something is not significant alone, or it is significant even under the Bonferonni correction :

In [ ]:
(air_poll_granger.alone_sig == air_poll_granger.bonf_sig).all()

In other words, there is no need for a finer correction like Benjamin-Hochberg FDR.

We find that for most countries, past wikipedia views make for a good linear predictor of the future of air pollution. This even holds for Turkey where the air pollution got worse during Covid.

An interesting case is that of Japan and South Korea which have very insignificant p-values (>.2), suggesting that day-to-day linear prediction is not very convincing for these two countries. We note that these are the only Eastern-Asian countries in the dataset. An interesting extension to this project could be to check whether this extends further to other countries in the area.

The odd one out is then Serbia, with a p-value of 0.21. There is an explanation which we find satisfying : while air pollution follows the same patterns as similar European countries, the wikipedia views on the environment topic in Serbia are fairly unusual when compared to neighboring countries. Indeed, there is a massive spike around 2020-04, the time at which Covid hit the country. Previously, the article had significnatly fewer views. For most other countries, the process was much more continuous. We see no particular reason as to why the awareness differs, but this explains why the Granger test sees no significant use of wikipedia views for predicting the air pollution.

For all others, the linear prediction works out fine. The model is confident that the past of the wikipedia views is a useful tool to predict the air pollution of the next day in the capital.

# Part 4 : What if ? and what happens next ?

In this final part, we want to create a hypothetical scenario of 2022 using statistical forecasting without the data from that year. The idea is to show whether the direction air pollution is taking is predictable, and where it is headed.

For the statistical forecasting, we will use the [SARIMA model](https://en.wikipedia.org/wiki/Autoregressive_integrated_moving_average#Variations_and_extensions) which enables the prediction of the future of a timeseries by using the previous data points and accounting for seasonality.

We will analyze the data in the following way : we will predict whether the air pollution in the period of 2022 can be meaningfully predicted from the previous years.
Ideally, the plan was to attempt to predict what would have heppened by training the model on air data from 2015 to 2019 and predicting 2020-2022, in order to compare with what actually happened.
It turns out, the air data is not available so long in the past. We had to modify the analysis to analyze from 2019-2021 and predict 2022, to give the model data on which it could be trained.
This modifies a bit the intent of the analysis. Instead of seeing whether there is a downwards perturbance in air pollution (which we already know happened), we now ask a slightly different question : "does the air pollution in 2022 significantly change from the trends of 2019-2021, and if so, what is the direction of the change?". This will give us insights into what the different countries have "learned". For example, a year 2022 that is unpredictably low in terms of air pollution means that the country has (for now) learned that it could survive without as much pollution.

There could be other explanations (the country never economically recovered from Covid, ...) but there is little we can do to account for this in time. Besides, speaking only in terms of air pollution, the conclusion is the same : the country is for the foreseeable future on its way to have a better air.

In [ ]:
sarima_list = []
for country in country_codes : 
    try:
        print("=========================================================")
        print(country, "\n") 
        
        dataset = wikipedia_language_to_dataset[country][1].copy()
        dataset = dataset[dataset["dt"].dt.year <= 2021]
        dataset = dataset.set_index("dt")
        dataset = dataset["median"]
        decomposition = sm.tsa.seasonal_decompose(dataset.resample('MS').mean(), model='additive')
        fig = decomposition.plot()
        plt.show()

        mod = sm.tsa.statespace.SARIMAX(dataset.resample('MS').mean(),
                                        order=(0, 1, 1),
                                        seasonal_order=(1, 1, 1, 12))
        results = mod.fit(method = 'powell')

        pred = results.get_prediction(start=pd.to_datetime('2022-01-01'), end = pd.to_datetime('2023-01-01'), dynamic=False)
        pred.predicted_mean.plot()
        dataset = wikipedia_language_to_dataset[country][1].copy()
        dataset = dataset.set_index("dt")
        dataset = dataset["median"]
        dataset = dataset.rename("actual_mean")
        dataset.resample('MS').mean().plot()
        plt.legend()
        plt.show()
        
        dataset_2022 = wikipedia_language_to_dataset[country][1]
        dataset_2022 = dataset_2022[dataset_2022["dt"].dt.year > 2021]
        dataset_2022 = dataset_2022.set_index("dt")
        dataset_2022 = dataset_2022["median"].resample('MS').mean()
        
        joined_true_and_pred = pd.concat([pred.predicted_mean, dataset_2022], axis = 1, join = 'inner')
        mae = mean_absolute_error(joined_true_and_pred["median"], joined_true_and_pred["predicted_mean"])
        
        expectedMean = dataset_2022.mean()
        predMean = pred.predicted_mean.mean()
        relative_diff = (
            "{:+.2f}".format(
                ((predMean / expectedMean) - 1) * 100
            )+ "%"
        )
        sarima_list.append((country, expectedMean, predMean, relative_diff, mae))
        print("\n\n\n")
    except:
        print("not enough data for sarima!")
        continue

In [ ]:
sarimaDf = pd.DataFrame(sarima_list, columns = ["Country", "expected", "prediction", "relative_diff", "mae"])

In [ ]:
sarimaDf

Here, we have fairly binary results : 
- Either the country has an actual pollution that is lower than the prediction (Japan, Turkey, Norway, the U.S., Germany, France, Korea, Finland). The country that has the largest MAE is in this category: it is Turkey, which has a massive drop in air pollution in 2022, while its trend was increasing before that year.
- Or it has an actual pollution that is higher than the prediction (Italy, Serbia, Sweden, the Netherlands, Catalonia). In this case, the largest difference between the actual pollution is Sweden, which is explained by noticing that the values for Sweden are usually really low, and that there is an small, unexplained peak in the air pollution in 2022. This is seen in the Sarima trends graph of the country. Most countries in this category behave this way.

We also note that Denmark is the only country with a full month of data missing, so it is not included in this study. 

We can conclude in the following way :
For most countries, it holds that either the model predicts a higher pollution than reality for 2022 or the country emissions were already fairly low. This, in turn, suggests that humans were indeed taught a lesson by Covid in terms of climate change.